In [ ]:
!hdfs dfs -rm -r -skipTrash .Trash
!hdfs dfs -rm -r -skipTrash 01_sparkStaging
!hdfs dfs -rm -r -skipTrash .sparkStaging

In [ ]:
!pip install workadays

In [ ]:
!pip install samba pysmb

In [ ]:
import numpy as np
import pandas as pd
import os
import logging

from funcoes_painel_acomp.painelAcomp import marca_status, painel_resumo_processos, dash_dq_prod

import tempfile
from smb.SMBConnection import SMBConnection
import base64

import holidays as hl
from datetime import datetime
from datetime import timedelta
from workadays import workdays as wd
from calendar import monthrange

from random import random
from operator import add

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import types as t

from pyspark.sql import functions as f
from pyspark.sql.functions import datediff
from pyspark.sql.functions import format_number
from pyspark.sql.functions import row_number
from pyspark.sql.functions import trim
from pyspark.sql.functions import lpad
from pyspark.sql.functions import abs
from pyspark.sql.types import DecimalType, StructType, StructField, StringType, IntegerType

from pyspark.sql.window import *
from dateutil.relativedelta import *

br_holidays = hl.CountryHoliday('BR')

In [ ]:
spark = SparkSession\
.builder\
.appName('tb_prod')\
.enableHiveSupport()\
.config("spark.executor.memory", "16G")\
.config("spark.executor.memoryOverhead","2G")\
.config("spark.executor.cores", "9")\
.config("spark.dynamicAllocation.enabled", "true")\
.config("spark.dynamicAllocation.initialExecutors", "1")\
.config("spark.dynamicAllocation.maxExecutors","12")\
.config("spark.dynamicAllocation.minExecutors","1")\
.config("spark.driver.memory", "16G")\
.config("spark.driver.memoryOverhead", "2G")\
.config("spark.driver.maxResultSize", "16G")\
.config("spark.sql.broadcastTimeout", "36000")\
.config("spark.sql.hive.convertMetastoreParquet","false")\
.config("spark.hadoop.hive.metastore.client.socket.timeout", "900")\
.config("spark.network.timeout", "8000")\
.config("spark.memory.fraction","0.6")\
.config("spark.shuffle.service.enabled", "true")\
.config("spark.ui.killEnabled", "true")\
.config("spark.rpc.message.maxSize","1024")\
.config("spark.kryoserializer.buffer.max", "2000mb")\
.config('spark.yarn.queue', 'root.gapl_planejamentos_incentivos')\
.config('hive.exec.dynamic.partition.mode',"nonstrict")\
.getOrCreate()

In [ ]:
###########################################################
#############      Automatização de Datas     #############
###########################################################

dt_today = datetime.today().date()
#dt_today = datetime(2023, 6, 5).date();

n_week = dt_today.weekday()
f_hly = dt_today in br_holidays

dt_prodm3 = wd.workdays(dt_today, -3, country='BR') ## dt_anomes
dt_prodm1 = wd.workdays(dt_today, -1, country='BR') ## dt_pe + dt_cver_movi
dt_posi = wd.workdays(dt_today, -1, country='BR') ## dt_pe + dt_cver_movi

dt_mes_ant = (dt_posi - relativedelta(months=1))
dt_anomes_aaaamm_ant = dt_mes_ant.strftime('%Y%m')

dt_anomes_aaaamm = dt_prodm3.strftime('%Y%m')
dt_anomes_aaaa_mm = dt_prodm3.strftime('%Y-%m') 

dt_seg = wd.workdays(dt_today, -35, country='BR')
dt_seg_aamm = dt_prodm3.strftime('%y%m')

dt_cart = wd.workdays(dt_today, -30, country='BR')
dt_cart_aaaamm = dt_prodm3.strftime('%Y%m')

dt_nme = dt_prodm1.strftime('%Y%m%d')
dt_nme_mens = dt_mes_ant.strftime('%Y%m')
dt_nme3 = dt_prodm3.strftime('%Y%m%d')
dt_anomes_nme = dt_prodm3.strftime('%Y%m')

print('Dia da Execução ->',dt_today)
print('Data da Producao ->',dt_prodm1)
print('Data da segmentação - >', dt_prodm3)
print('dt_anomes ->',dt_anomes_aaaamm)
print('anomes ->',dt_anomes_aaaa_mm)
print('Data De_Para de Segmento ->',dt_seg_aamm)
print('Data carterizacao ->',dt_cart_aaaamm)
print('Data nme ->',dt_nme)
print('Data nme3 ->',dt_nme3)
print('Anomes nme ->',dt_anomes_nme)
print('dt_nme_mens ->',dt_nme_mens)

In [ ]:
dia_util = spark.sql(""" select du_br from sand_invest.tb_du where replace(data,'-','') = {} """.format(dt_nme)).collect()[0][0]
print('dia util ->', dia_util)

In [ ]:
dt_estr = spark.sql(""" select max(dat_ref_carga) from sand_invest.tb_bis_dat_ref_estr where dt_anomes = {} """.format(dt_anomes_aaaamm)).collect()[0][0]
if dt_estr == None:
    dt_estr = spark.sql(""" select max(dat_ref_carga) from sand_invest.tb_bis_dat_ref_estr where dt_anomes = {} """.format(dt_anomes_aaaamm_ant)).collect()[0][0]
    print(f'Estrutura: {dt_anomes_aaaamm_m0} vazia! Pegando a maxima data disponivel ->', dt_estr)
else:
    dt_estr
    print('Estrutura ->', dt_estr)

In [ ]:
if (int(dia_util) >= 1) and (int(dia_util) < 3):
    dt_segm = dt_prodm3
    print('Periodo de fechamento:')
    print('  data segmento ->', dt_segm)
else:
    dt_segm = dt_prodm1
    print('Fluxo normal:')
    print('  data segmento ->', dt_segm)

In [ ]:
if (int(dia_util) >= 1) and (int(dia_util) < 3):
    dt_prod_m0 = wd.workdays(dt_today, -1, country='BR')
    dt_anomes_aaaamm_m0 = dt_today.strftime('%Y%m')
    dt_anomes_aaaa_mm_m0 = dt_today.strftime('%Y-%m')
    print('Periodo de fechamento:')
    print('  Mês fechamento:')
    print('    Data de produção ->', dt_prodm1)
    print('    dt_anomes ->', dt_anomes_aaaamm)
    print('    Anomes ->', dt_anomes_aaaa_mm)
    print('  Mês atual:')
    print('    Data de produção m0 ->', dt_prod_m0)
    print('    dt_anomes m0 ->', dt_anomes_aaaamm_m0)
    print('    Ano-mes m0 ->', dt_anomes_aaaa_mm_m0)
    
else:
    dt_prod_m0 = dt_prodm1
    dt_anomes_aaaamm_m0 = dt_anomes_aaaamm
    dt_anomes_aaaa_mm_m0 = dt_anomes_aaaa_mm
    print('Fluxo normal:')
    print(' Data de produção ->', dt_prodm1)
    print(' dt_anomes ->', dt_anomes_aaaamm)
    print(' Anomes ->', dt_anomes_aaaa_mm)

In [ ]:
###########################################################
################       Grava Tabelas       ################
################    DQ - Step Controle     ################
###########################################################
################        - Inicio -         ################
###########################################################

spark.sql("ALTER TABLE sand_invest.tb_dq_qlik_wf DROP IF EXISTS PARTITION(ds_processo = 'prod', dat_ref_carga = '{}')".format(dt_today))

dt_exec_ini = datetime.today()

info_dq_wf_ini = [('prod','Iniciado',str(dt_exec_ini),'0','0',str(dt_prodm1),str(dt_anomes_aaaa_mm),str(dt_today))]

schema_dq_wf = StructType([StructField("ds_processo",StringType(),True), \
                           StructField("status_carga",StringType(),True), \
                           StructField("time_carga",StringType(),True), \
                           StructField("delta_time_carga", StringType(), True), \
                           StructField("qtd_reg_dup", StringType(), True), \
                           StructField("dt_posi", StringType(), True), \
                           StructField("dt_anomes", StringType(), True), \
                           StructField("dat_ref_carga", StringType(), True) \
                          ])

dq_wf_ini = spark.createDataFrame(data=info_dq_wf_ini,schema=schema_dq_wf)

dq_wf_ini.write.mode("append").partitionBy('ds_processo','dt_anomes','dat_ref_carga').saveAsTable('sand_invest.tb_dq_qlik_wf')

In [ ]:
###########################################################
##############        Bases Auxiliares       ##############
###########################################################

#Cadastral de Clientes
pe001 = spark.read.table('pe.pedt001')\
             .filter(f.col('dat_ref_carga') == dt_segm)\
             .select('penumper','penomper','petipper',\
                     'pesexper','pefecnac','pesubseg')\
             .dropDuplicates()

#Cadastral de Clientes - Tabela de Documentos(13 -> CPF, 14 -> CNPJ)
pe150 = spark.read.table('pe.pedt150')\
             .filter((f.col('dat_ref_carga') == dt_segm) &\
                     (f.col('petipdoc').isin('13','14')))\
             .select('penumper','petipdoc','penumero','pefecalt')\
             .withColumn('rank', f.row_number().over(Window.partitionBy('penumper').orderBy(f.col('petipdoc').desc(),f.col('pefecalt').desc())))\
             .filter(f.col('rank') == 1)

#Depara de Segmento(analizar fluxo para consumir do datashare de riscos(Bruno Visnadi))
segm = spark.read.table('sand_invest.bu_ifrs9_depara_segm_2109')
    
#Cadastral de Clientes + Descrição de Segmento 
pe001_seg = pe001.join(pe150, pe001['penumper'] ==  pe150['penumper'], 'left')\
                 .join(segm, pe001['pesubseg'].cast('int') == segm['cod_segmento'].cast('int'), 'left')\
                 .select(pe001['*'],\
                         pe150['petipdoc'],\
                         pe150['penumero'].cast(DecimalType(20, 0)),\
                         segm['*'])

#Base de Clientes - Time Investimentos(processo interno)
tbCli = spark.read.table('sand_invest.tb_clientes_lake_new')\
             .filter(f.col('dt_anomes') == dt_anomes_aaaamm)

#Descritivo dos produtos
do003 = spark.read.table('do.dodt003')\
             .withColumn('rank_prod', f.row_number().over(Window.partitionBy('sg_sist','cd_prod','cd_subp').orderBy(f.col('dat_ref_carga').desc())))\
             .filter(f.col('rank_prod') == 1)

#Depara canal
cnal = spark.read.table('sand_invest.tb_depara_canal')

#Base com dados de produtos - infos de risco de produto - área do Marquinhos (produtos) precisa nos passar (analisar se não temos   essas   infos na do003)
tbOq = spark.read.table('sand_invest.tb_oq_v5')\
            .filter(f.regexp_replace('sistema_origem',' ','').isin('BG','BP','EY','MX','KPV','SBK','NW'))\
            .select(f.regexp_replace('sistema_origem',' ','').alias('sistema_origem'),\
                    f.regexp_replace(f.col('cd_prod'),' ','').alias('cd_prod'),\
                    f.regexp_replace(f.col('cd_subp'),' ','').alias('cd_subp'),\
                    f.col('classe_ativo'),\
                    f.col('cd_situacao'),\
                    f.regexp_replace(f.col('req'),',','.').alias('req'),\
                    f.col('liquidez'),\
                    f.col('risco'),\
                    f.col('anomes'))

#Base de Produtos Diversificação - Contem marcação se produto é considerado como diversificado ou não
tbProdDiv = spark.read.table('sand_invest.tb_diversificacao')

#Base de Vencimentos RF
tbVenc = spark.read.table('do.dodt202_vrj')\
              .select(f.col('cd_clie'),\
                      f.col('cd_cntr'),\
                      f.col('dt_venc_oper')\
                      )\
              .withColumn('rank_venc',f.row_number().over(Window.partitionBy('cd_clie','cd_cntr').orderBy(f.col('dt_venc_oper').desc())))\
              .filter(f.col('rank_venc') == 1)

In [ ]:
                                        ###########################################################
                                        ###########################################################
                                        ##############       Bases de Produtos       ##############
                                        ###########################################################
                                        ###########################################################

###########################################################
##############       Bases de Produtos       ##############
##############          PREVIDENCIA          ##############
########################################################### 

do207 = spark.read.table('do.dodt207')\
             .filter((f.col('dt_cver_movi') <= dt_prodm1) &\
                     (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_brut') == 0)\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt207'))

do207_vrj = spark.read.table('do.dodt207_vrj')\
             .filter((f.col('dt_cver_movi') <= dt_prodm1) &\
                     (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_brut') == 0)\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt207_vrj'))

##################################################
#### PERIODO DE FECHAMENTO - PEGANDO M0 E M-1 ####
##################################################

if (int(dia_util) >= 1) and (int(dia_util) < 3):
    do207_m0 = spark.read.table('do.dodt207')\
                 .filter((f.col('dt_cver_movi') <= dt_prod_m0) &\
                         (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_brut') == 0)\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt207'))
    
    do207_vrj_m0 = spark.read.table('do.dodt207_vrj')\
                 .filter((f.col('dt_cver_movi') <= dt_prod_m0) &\
                         (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_brut') == 0)\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt207_vrj'))

    do207_full = do207.union(do207_vrj)\
                        .union(do207_m0)\
                        .union(do207_vrj_m0)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt',\
                                                                                 'cd_subp_prvd','cd_pape_prvd','nr_cerf_prvd',\
                                                                                 'nr_sequ_movi','dt_cver_movi')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

else:
    do207_full = do207.union(do207_vrj)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt',\
                                                                                 'cd_subp_prvd','cd_pape_prvd','nr_cerf_prvd',\
                                                                                 'nr_sequ_movi','dt_cver_movi')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

do207_prod = do207_full.select(f.col('dt_movi').alias('dt_movi'),\
                               f.col('cd_clie').alias('nr_pess'),\
                               f.col('cd_banc').alias('cd_banc'),\
                               f.col('cd_agen').alias('cd_agen'),\
                               f.col('nr_cnta_crrt').alias('nr_cnta_crrt'),\
                               f.col('nr_cerf_prvd').alias('cd_cntr'),\
                               f.lit('0').alias('tx_clie'),\
                               f.lit('0').alias('tx_tti'),\
                               f.lit('0').alias('tx_spread'),\
                               f.lit(' ').alias('ds_indx'),\
                               f.col('nr_sequ_movi').alias('nr_sequ_movi'),\
                               f.col('cd_pape_prvd').alias('cd_fundo'),\
                               f.col('ds_pape').alias('ds_fundo'),\
                               f.lit('9999-99-99').alias('dt_vencimento'),\
                               f.when(f.regexp_replace(f.col('cd_pape_prvd'),' ','').isin('1016','1022','1050','1056','1060','8000','1016A','9999'),'0')\
                                .when((f.regexp_replace(f.col('cd_pape_prvd'),' ','') == '8001') & \
                                      (f.regexp_replace(f.col('cd_subp_tc'),' ','').isin('2333','4580','4583','4587','6244','6245','6246','6247','6248',\
                                                                                             '6249','6250','6251','6252','6253','6254','6255','6256','6257',\
                                                                                             '6258','6259','6260','6280','6281','6282','6283','6284','6285',\
                                                                                             '6286','6287','6288','6289','6308','6309','6310','6311','6312',\
                                                                                             '6313','6318','6319')),'0')\
                                .when(f.regexp_replace(f.col('cd_subp_tc'),' ','') == '3707','0')\
                                .when(f.regexp_replace(f.col('cd_subp_tc'),' ','') == '08000','0')\
                                .when(f.regexp_replace(f.col('in_movi_traf'),' ','') == 'S','0')\
                                .when(((f.col('cd_subp_prvd').cast('double') >= 3000) &\
                                      (~(f.col('cd_subp_prvd').cast('double').isin(  5200,	5201,	5202,	5203,	5204,	5205,	5206,	5207,	5210,	5211,
                                                                                    5212,	5213,	5214,	5217,	5218,	5219,	5220,	5222,	5223,	5224,
                                                                                    5225,	5226,	5227,	5229,	5230,	5231,	5232,	5233,	5234,	5235,
                                                                                    5236,	5237,	5238,	5239,	5241,	5242,	5245,	5246,	5247,	5251,
                                                                                    5252,	5253,	5254,	5255,	5256,	5259,	5260,	5261,	5262,	5264,
                                                                                    5265,	5266,	5267,	5268,	5269,	5270,	5271,	5272,	5273,	5276,
                                                                                    5277,	5278,	5279,	5281,	5282,	5283,	5284,	5285,	5286,	5287,
                                                                                    5288,	5289,	5290,	5292,	5294,	5295,	5296,	5297,	5298,	5299,
                                                                                    5300,	5301,	5302,	5303,	5304,	5305,	5306,	5307,	5309,	5400,
                                                                                    5401,	5402,	5403,	5404,	5405,	5406,	5407,	5408,	5409,	5411,
                                                                                    5412,	5413,	5414,	5415,	5416,	5417,	5418,	5419,	5420,	5422,
                                                                                    5423,	5424,	5425,	5426,	5427,	5428,	5429,	5431,	5432,	5434,
                                                                                    5435,	5436,	5438,	5439,	5440,	5441,	5442,	5444,	5445,	5447,
                                                                                    5448,	5449,	5450,	5451,	5452,	5453,	5454,	5455,	5456,	5457,
                                                                                    5458,	5459,	5460,	5461,	5462,	5463,	5464,	5465,	5466,	5467,
                                                                                    5468,	5469,	5470,	5471,	5472,	5473,	5474,	5475,	5476,	5477,
                                                                                    5479,	5480,	5481,	5482,	5483,	5484,	5485,	5486,	5487,	5488,
                                                                                    5489,	5490,	5491,	5492,	5493)))),'00')\
                                .when((f.regexp_replace(f.col('tp_movi_prtb'),' ','') == 'E') |\
                                      (f.col('ds_tipo_movi').like ('%Portabilidade%')),'1')\
                                .when(f.col('ds_tipo_movi').like('%Resgate Total%'),'1')\
                                .when(f.col('ds_tipo_movi').like('%Resgate Parcial%'),'1')\
                                .when(f.col('ds_tipo_movi').like('%Aporte%'),'1')\
                                .when(f.col('ds_tipo_movi').like('%Pagamento Saldo da Reserva Obito%'),'0')\
                                .when(f.col('ds_tipo_movi').like('%Lançamento de Rentabilidade%'),'0')\
                                .when(f.col('ds_tipo_movi').like('%Excedente Financeiro%'),'0')\
                                .when(f.col('ds_tipo_movi').like('%Juros%'),'0')\
                                .when(f.col('ds_tipo_movi').like('%Concessao/Pagamento de Beneficio%'),'0')\
                                .when(f.col('ds_tipo_movi').like('%Contribuição%'),'1')\
                                .otherwise('0').alias('fl_prod'),\
                               f.lit('0').alias('fl_fundo_auto'),\
                               f.lit('0').alias('fl_fundo_espelho'),\
                               f.when((f.col('tp_movi_prtb') == 'E') | (f.col('ds_tipo_movi').like('%Portabilidade%')),'1')\
                                .otherwise('0').alias('fl_portabilidade'),\
                               f.lit('A0').alias('cd_prod'),\
                               f.regexp_replace(f.col('cd_subp_prvd'),' ','').alias('cd_subp'),\
                               f.regexp_replace(f.col('cd_subp_tc'),' ','').alias('cd_subp_tc'),\
                               f.col('dat_ref_carga').alias('dat_ref_carga'),\
                               f.col('dt_cver_movi').alias('dt_cver_movi'),\
                               f.trim(f.col('cd_movi_sist_orig')).alias('cd_tipo_movi'),\
                               f.trim(f.col('ds_tipo_movi')).alias('ds_tipo_movi'),\
                               f.when(((f.col('tp_movi_inve') == 'E') & ~(f.upper(f.col('ds_tipo_movi')).like('%ESTOR%'))) |\
                                      ((f.col('tp_movi_inve') == 'S') & (f.upper(f.col('ds_tipo_movi')).like('%ESTOR%'))),'1')\
                                .when(((f.col('tp_movi_inve') == 'S') & ~(f.upper(f.col('ds_tipo_movi')).like('%ESTOR%'))) |\
                                      ((f.col('tp_movi_inve') == 'E') & (f.upper(f.col('ds_tipo_movi')).like('%ESTOR%'))),'-1')\
                                .alias('tp_movi'),\
                               f.when(f.col('tp_movi_inve') == 'E', f.col('vl_brut'))\
                                .otherwise(f.col('vl_brut') * (-1)).alias('vl_movi'),\
                               f.lit(' ').alias('cd_canal'),\
                               f.lit(' ').alias('ds_canal'),\
                               f.col('cd_usua_oper').alias('cd_ang'),\
                               f.lit(' ').alias('ds_mod_atend'),\
                               f.lit('0').alias('fl_diversificacao'),\
                               f.concat(f.col('dt_cver_movi').substr(1,4),\
                                        f.col('dt_cver_movi').substr(6,2)).alias('dt_anomes'),\
                               f.lit('PREVIDENCIA').alias('ds_familia_carga'),\
                               f.regexp_replace(f.concat(f.coalesce(f.col('cd_banc'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_agen'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_subp_prvd'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_pape_prvd'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('nr_cnta_crrt'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('nr_cerf_prvd'), f.lit('9999'))),' ','').alias('pk_key'),\
                               f.lit('KPV').alias('sistema_origem'),\
                               f.col('tb_origem').alias('tb_origem')\
                              )

###########################################################
##############       Bases de Produtos       ##############
##############            FUNDOS             ##############
###########################################################

do205 = spark.read.table('do.dodt205')\
             .filter((f.col('dt_efet_movi') <= dt_prodm1) &\
                     (f.col('dt_efet_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_brut') == 0)\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt205'))

do205_vrj = spark.read.table('do.dodt205_vrj')\
             .filter((f.col('dt_efet_movi') <= dt_prodm1) &\
                     (f.col('dt_efet_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_brut') == 0)\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt205_vrj'))

##################################################
#### PERIODO DE FECHAMENTO - PEGANDO M0 E M-1 ####
##################################################

if (int(dia_util) > 1) and (int(dia_util) <= 3):
    do205_m0 = spark.read.table('do.dodt205')\
                 .filter((f.col('dt_efet_movi') <= dt_prod_m0) &\
                         (f.col('dt_efet_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_brut') == 0)\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt205'))
    
    do205_vrj_m0 = spark.read.table('do.dodt205_vrj')\
                 .filter((f.col('dt_efet_movi') <= dt_prod_m0) &\
                         (f.col('dt_efet_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_brut') == 0)\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt205_vrj'))

    do205_full = do205.union(do205_vrj)\
                        .union(do205_m0)\
                        .union(do205_vrj_m0)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt','cd_clie_cntr',\
                                                                                 'nr_cerf_fund','nr_sequ_movi_fund','dt_efet_movi','nr_oper','tp_movi_mx')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

else:
    do205_full = do205.union(do205_vrj)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt','cd_clie_cntr',\
                                                                                 'nr_cerf_fund','nr_sequ_movi_fund','dt_efet_movi','nr_oper','tp_movi_mx')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

## ERRO NOS DADOS DE RESGATE DO DIA 23-05

if (dt_anomes_aaaa_mm == '2022-05'):
    
    cot_fdo = spark.read.table('sand_invest.tb_cota_fundo')
    
    do205_join = do205_full.join(do003, (f.lit('MX') == do003['sg_sist']) &\
                                    (f.lpad(f.regexp_replace(do205_full['cd_subp_fund'],' ',''),10,'0') == \
                                     f.lpad(f.regexp_replace(do003['cd_subp'],' ',''),10,'0')), 'left')\
                       .join(cot_fdo, (cot_fdo['cd_subp_fund'].cast('double') == do205_full['cd_subp_fund'].cast('double')) &\
                                      (cot_fdo['vl_cota_fund'].cast('double') == do205_full['vl_cota_fund'].cast('double')),'left')\
                            .select(do205_full['*'],\
                                    f.when(cot_fdo['cd_subp_fund'].isNull(),'0')\
                                     .otherwise('1').alias('erro'),\
                                    do003['cd_subp_tc'])\
                            .filter(f.col('erro') == '0')\
    
else :
    do205_join = do205_full.join(do003, (f.lit('MX') == do003['sg_sist']) &\
                                        (f.lpad(f.regexp_replace(do205_full['cd_subp_fund'],' ',''),10,'0') == \
                                         f.lpad(f.regexp_replace(do003['cd_subp'],' ',''),10,'0')), 'left')\
                            .select(do205_full['*'],\
                                    do003['cd_subp_tc'])

do205_prod = do205_join.select(f.col('dt_movi').alias('dt_movi'),\
                               f.col('cd_clie').alias('nr_pess'),\
                               f.col('cd_banc').alias('cd_banc'),\
                               f.col('cd_agen').alias('cd_agen'),\
                               f.col('nr_cnta_crrt').alias('nr_cnta_crrt'),\
                               f.col('nr_cerf_fund').alias('cd_cntr'),\
                               f.col('pc_taxa_admi').alias('tx_clie'),\
                               f.lit(' ').alias('tx_tti'),\
                               f.col('pc_taxa_admi').alias('tx_spread'),\
                               f.lit(' ').alias('ds_indx'),\
                               f.col('nr_sequ_movi_fund').alias('nr_sequ_movi'),\
                               f.col('cd_subp_fund').alias('cd_fundo'),\
                               f.lit(' ').alias('ds_fundo'),\
                               f.lit('9999-99-99').alias('dt_vencimento'),\
                               f.when((f.col('cd_subp_fund').cast('int').isin(3530,674,675,1548,416)) |\
                                      (f.col('tp_movi_mx').cast('int').isin(45,55)) |\
                                      ((f.col('tp_movi_mx').cast('double').isin(21))&(f.col('cd_subp_fund').cast('double')==4351)) |\
                                      (f.trim(f.col('ds_tipo_movi_mx')) == 'ANTECIPACAO IR'),'0')\
                                .otherwise('1').alias('fl_prod'),\
                               f.when(f.col('cd_subp_fund').cast('int').isin(3530,674,675,1548,416),'1')\
                                .otherwise('0').alias('fl_fundo_auto'),\
                               f.lit('0').alias('fl_fundo_espelho'),\
                               f.lit('0').alias('fl_portabilidade'),\
                               f.lit('35').alias('cd_prod'),\
                               f.regexp_replace(f.col('cd_subp_fund'),' ','').alias('cd_subp'),\
                               f.regexp_replace(f.col('cd_subp_tc'),' ','').alias('cd_subp_tc'),\
                               f.col('dat_ref_carga').alias('dat_ref_carga'),\
                               f.col('dt_efet_movi').alias('dt_cver_movi'),\
                               f.trim(f.col('tp_movi_mx')).alias('cd_tipo_movi'),\
                               f.trim(f.col('ds_tipo_movi_mx')).alias('ds_tipo_movi'),\
                               f.when(((f.col('tp_movi_inve') == 'E') & ~(f.upper(f.col('ds_tipo_movi_mx')).like('%ESTOR%'))) |\
                                      ((f.col('tp_movi_inve') == 'S') & (f.upper(f.col('ds_tipo_movi_mx')).like('%ESTOR%'))),'1')\
                                .when(((f.col('tp_movi_inve') == 'S') & ~(f.upper(f.col('ds_tipo_movi_mx')).like('%ESTOR%'))) |\
                                      ((f.col('tp_movi_inve') == 'E') & (f.upper(f.col('ds_tipo_movi_mx')).like('%ESTOR%'))),'-1')\
                                .alias('tp_movi'),\
                               f.when(f.col('tp_movi_inve') == 'E', f.col('vl_brut'))\
                                .otherwise(f.col('vl_brut')*(-1)).alias('vl_movi'),\
                               f.col('cd_cnal_oper').alias('cd_canal'),\
                               f.lit(' ').alias('ds_canal'),\
                               f.col('cd_usua_oper').alias('cd_ang'),\
                               f.lit(' ').alias('ds_mod_atend'),\
                               f.lit('0').alias('fl_diversificacao'),\
                               f.concat(f.col('dt_efet_movi').substr(1,4),\
                                        f.col('dt_efet_movi').substr(6,2)).alias('dt_anomes'),\
                               f.lit('FUNDOS').alias('ds_familia_carga'),\
                               f.regexp_replace(f.concat(f.coalesce(f.col('cd_banc'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_agen'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('nr_cnta_crrt'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_clie_cntr'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('nr_cerf_fund'), f.lit('9999'))),' ','').alias('pk_key'),\
                               f.lit('MX').alias('sistema_origem'),\
                               f.col('tb_origem').alias('tb_origem')\
                              )

###########################################################
##############       Bases de Produtos       ##############
##############          RENDA FIXA           ##############
###########################################################

do203 = spark.read.table('do.dodt203')\
             .filter((f.col('dt_cver_movi') <= dt_prodm1) &\
                     (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.regexp_replace(f.col('ds_tipo_movi_impa'),' ','') == 'TRANSFERENCIA') &\
                     ~(f.col('vl_brut_movi') == 0) &\
                     ~(f.trim(f.col('ds_fami_prod')) == 'DRA')\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt203'))

do203_vrj = spark.read.table('do.dodt203_vrj')\
             .filter((f.col('dt_cver_movi') <= dt_prodm1) &\
                     (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.regexp_replace(f.col('ds_tipo_movi_impa'),' ','') == 'TRANSFERENCIA') &\
                     ~(f.col('vl_brut_movi') == 0) &\
                     ~(f.trim(f.col('ds_fami_prod')) == 'DRA')\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt203_vrj')) 

##################################################
#### PERIODO DE FECHAMENTO - PEGANDO M0 E M-1 ####
##################################################

if (int(dia_util) > 1) and (int(dia_util) <= 3):
    do203_m0 = spark.read.table('do.dodt203')\
                 .filter((f.col('dt_cver_movi') <= dt_prod_m0) &\
                         (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.regexp_replace(f.col('ds_tipo_movi_impa'),' ','') == 'TRANSFERENCIA') &\
                         ~(f.col('vl_brut_movi') == 0) &\
                         ~(f.trim(f.col('ds_fami_prod')) == 'DRA')\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt203'))

    do203_vrj_m0 = spark.read.table('do.dodt203_vrj')\
                 .filter((f.col('dt_cver_movi') <= dt_prod_m0) &\
                         (f.col('dt_cver_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.regexp_replace(f.col('ds_tipo_movi_impa'),' ','') == 'TRANSFERENCIA') &\
                         ~(f.col('vl_brut_movi') == 0) &\
                         ~(f.trim(f.col('ds_fami_prod')) == 'DRA')\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt203_vrj')) 

    do203_full = do203.union(do203_vrj)\
                        .union(do203_m0)\
                        .union(do203_vrj_m0)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt',
                                                                                 'cd_cntr','dt_cver_movi','tp_impa_inve')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

else:
    do203_full = do203.union(do203_vrj)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt',
                                                                                 'cd_cntr','dt_cver_movi','tp_impa_inve')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

do203_prod = do203_full.select(f.col('dt_movi').alias('dt_movi'),\
                               f.col('cd_clie').alias('nr_pess'),\
                               f.col('cd_banc').alias('cd_banc'),\
                               f.col('cd_agen').alias('cd_agen'),\
                               f.col('nr_cnta_crrt').alias('nr_cnta_crrt'),\
                               f.col('cd_cntr').alias('cd_cntr'),\
                               f.when(f.regexp_replace(f.col('ds_indx'),' ','') == 'PRE', f.col('pc_taxa'))\
                                .otherwise(f.col('pc_indx')).alias('tx_clie'),\
                               f.lit(' ').alias('tx_tti'),\
                               f.lit(' ').alias('tx_spread'),\
                               f.regexp_replace(f.col('ds_indx'),' ','').alias('ds_indx'),\
                               f.lit('0').alias('nr_sequ_movi'),\
                               f.lit('0').alias('cd_fundo'),\
                               f.lit('0').alias('ds_fundo'),\
                               f.lit('9999-99-99').alias('dt_vencimento'),\
                               f.when(f.col('cd_subp_rf').cast('int').isin(555,556,557),'0')\
                                .otherwise('1').alias('fl_prod'),\
                               f.lit('0').alias('fl_fundo_auto'),\
                               f.lit('0').alias('fl_fundo_espelho'),\
                               f.lit('0').alias('fl_portabilidade'),\
                               f.lit('26').alias('cd_prod'),\
                               f.regexp_replace(f.col('cd_subp_rf'),' ','').alias('cd_subp'),\
                               f.lit('NA').alias('cd_subp_tc'),\
                               f.col('dat_ref_carga').alias('dat_ref_carga'),\
                               f.col('dt_cver_movi').alias('dt_cver_movi'),\
                               f.trim(f.col('tp_movi_impa_oper')).alias('cd_tipo_movi'),\
                               f.trim(f.col('ds_tipo_movi_impa')).alias('ds_tipo_movi'),\
                               f.when(f.trim(f.col('ds_tipo_movi_impa')).isin('APLICACAO','RENOVACAO'),'1')\
                                .when(f.trim(f.col('ds_tipo_movi_impa')).isin('VENCIMENTO','CANCELAMENTO'),'-1')\
                                .otherwise((f.when(f.trim(f.col('tp_impa_inve')) == 'E','1'))\
                                             .otherwise('-1')).alias('tp_movi'),\
                               f.when(f.trim(f.col('ds_tipo_movi_impa')).isin('APLICACAO','RENOVACAO'),abs(f.col('vl_brut_movi')))\
                                .when(f.trim(f.col('ds_tipo_movi_impa')).isin('VENCIMENTO','CANCELAMENTO'),abs(f.col('vl_brut_movi'))*(-1))\
                                .otherwise((f.when(f.trim(f.col('tp_impa_inve')) == 'E',abs(f.col('vl_brut_movi'))))\
                                             .otherwise(abs(f.col('vl_brut_movi'))*(-1))).alias('vl_movi'),\
                               f.col('cd_cnal_oper').alias('cd_canal'),\
                               f.lit(' ').alias('ds_canal'),\
                               f.col('cd_usua_oper').alias('cd_ang'),\
                               f.lit(' ').alias('ds_mod_atend'),\
                               f.lit('0').alias('fl_diversificacao'),\
                               f.concat(f.col('dt_cver_movi').substr(1,4),\
                                        f.col('dt_cver_movi').substr(6,2)).alias('dt_anomes'),\
                               f.lit('RENDA FIXA').alias('ds_familia_carga'),\
                               f.regexp_replace(f.concat(f.coalesce(f.col('cd_banc'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_agen'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('nr_cnta_crrt'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_cntr'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.concat(f.col('cd_banc'),f.col('cd_agen'),f.col('cd_cntr')), f.lit('9999'))),' ','').alias('pk_key'),\
                               f.lit('BP').alias('sistema_origem'),\
                               f.col('tb_origem').alias('tb_origem')\
                              )


###########################################################
##############       Bases de Produtos       ##############
##############           CORRETORA           ##############
###########################################################

do211 = spark.read.table('do.dodt211')\
             .filter((f.col('dt_liqu_crra') <= dt_prodm1) &\
                     (f.col('dt_liqu_crra').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_oper_crra') == 0) &\
                     ~(f.col('cd_clie') == '00000000') &\
                     (f.trim(f.col('tp_movi_crra')).isin('E','S'))\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt211'))


do211_vrj = spark.read.table('do.dodt211_vrj')\
             .filter((f.col('dt_liqu_crra') <= dt_prodm1) &\
                     (f.col('dt_liqu_crra').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_oper_crra') == 0) &\
                     ~(f.col('cd_clie') == '00000000') &\
                     (f.trim(f.col('tp_movi_crra')).isin('E','S'))\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt211_vrj'))

##################################################
#### PERIODO DE FECHAMENTO - PEGANDO M0 E M-1 ####
##################################################

if (int(dia_util) > 1) and (int(dia_util) <= 3):
    do211_m0 = spark.read.table('do.dodt211')\
                 .filter((f.col('dt_liqu_crra') <= dt_prod_m0) &\
                         (f.col('dt_liqu_crra').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_oper_crra') == 0) &\
                         ~(f.col('cd_clie') == '00000000') &\
                         (f.trim(f.col('tp_movi_crra')).isin('E','S'))\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt211'))
    
    do211_vrj_m0 = spark.read.table('do.dodt211_vrj')\
                 .filter((f.col('dt_liqu_crra') <= dt_prod_m0) &\
                         (f.col('dt_liqu_crra').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_oper_crra') == 0) &\
                         ~(f.col('cd_clie') == '00000000') &\
                         (f.trim(f.col('tp_movi_crra')).isin('E','S'))\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt211_vrj'))    
    
    do211_full = do211.union(do211_vrj)\
                        .union(do211_m0)\
                        .union(do211_vrj_m0)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc_crra','cd_agen_crra','nr_cnta_crrt_crra',
                                                                                 'cd_cntr_crra','cd_unic_info_crra','nr_movi_sequ_crra','dt_liqu_crra')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)
else:
    do211_full = do211.union(do211_vrj)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc_crra','cd_agen_crra','nr_cnta_crrt_crra',
                                                                                 'cd_cntr_crra','cd_unic_info_crra','nr_movi_sequ_crra','dt_liqu_crra')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

do211_prod = do211_full.select(f.col('dt_movi_crra').alias('dt_movi'),\
                               f.col('cd_clie').alias('nr_pess'),\
                               f.col('cd_banc_crra').alias('cd_banc'),\
                               f.col('cd_agen_crra').alias('cd_agen'),\
                               f.col('nr_cnta_crrt_crra').alias('nr_cnta_crrt'),\
                               f.col('cd_cntr_crra').alias('cd_cntr'),\
                               f.lit('0').alias('tx_clie'),\
                               f.lit(' ').alias('tx_tti'),\
                               f.lit(' ').alias('tx_spread'),\
                               f.col('cd_ativ_crra').alias('ds_indx'),\
                               f.col('nr_movi_sequ_crra').alias('nr_sequ_movi'),\
                               f.lit('0').alias('cd_fundo'),\
                               f.lit('0').alias('ds_fundo'),\
                               f.col('dt_venc_crra').alias('dt_vencimento'),\
                               f.when((f.trim(f.col('tp_orig_nego_crra')) == 'TED') &\
                                      (f.trim(f.col('ds_movi_prod_crra')).isin('TESOURO DIRETO - BLOQ JUD/ADM', 'TESOURO DIRETO - CANC CU JUROS',\
                                                                               'TESOURO DIRETO - COMPRA','TESOURO DIRETO - DEP/RET GARAN',\
                                                                               'TESOURO DIRETO - DEP/RET TIT G','TESOURO DIRETO - POS BLQ GA RE',\
                                                                               'TESOURO DIRETO - RESGATE','TESOURO DIRETO - TRANSFERENCIA',\
                                                                               'TESOURO DIRETO - VENDA')), '1')\
                                .when((f.trim(f.col('cd_subp_crra')).isin('CFC', 'CFE', 'CFO', 'COU',\
                                                                          'OPC', 'OPD', 'OPF', 'OPV', 'EOC',\
                                                                          'EOV', 'SWP', 'SWC', 'TFX', 'TNM'))&\
                                      (f.trim(f.col('ds_movi_prod_crra')).isin('DERIVATIVOS - FATUR BMF FUT','DERIVATIVOS - FATUR BMF FUT AJ',\
                                                                               'DERIVATIVOS - FATUR BMF DISP', 'ACOES - FATUR BOLSA OPT/EXERC',\
                                                                               'DERIVATIVOS - FATUR BMF OPCOES','DERIVATIVOS - FATUR BMF SWAP',\
                                                                               'DERIVATIVOS - FATUR BMF FLEX', 'ACOES  - FATUR BOLSA CONT TERM','ACOES - DEPOSITO DE ACOES','ACOES - TRANSF ENTRE CORRETORAS'\
                                                                               'ACOES - BOLSA LIQ CONT TERMO')),'1')\
                                .when((f.trim(f.col('cd_subp_crra')).isin('CTFI', 'TRV'))&\
                                      (f.trim(f.col('ds_movi_prod_crra')).like('PROVENTOS%')),'0')\
                                .when((f.trim(f.col('cd_subp_crra')).isin('CTFI', 'TRV'))&\
                                      (~(f.trim(f.col('ds_movi_prod_crra')).isin('TRANSFERENCIA DE C/C',\
                                                                                 'ACOES - TRANSF POR REGULARIZACAO','ACOES - AT.POR PGTO PROV',\
                                                                                 'ACOES - TRANSF POR REGULARIZACAO'))),'1')\
                                .when((f.trim(f.col('cd_subp_crra')).isin('BDR', 'ETFRF', 'ETFRV', 'RS', 'DS','ALT',
                                                                          'BS', 'CRI', 'DEB', 'CRA', 'CRV', 'ACN',
                                                                          'CDA', 'FIDC'))&\
                                      (f.trim(f.col('ds_movi_prod_crra')).isin('BOLSA - COMPRA','BOLSA - VENDA','ACOES - FATURAMENTO BOLSA COMP',\
                                                                               'ACOES - FATURAMENTO BOLSA VENDA', 'BOLSA - RESERVA DE ACAO',\
                                                                               'ACOES - RESERVA DE ACAO','ALGO+ TRANSF DE CARTEIRAS','ACOES - DEPOSITO DE ACOES','ACOES - TRANSF ENTRE CORRETORAS')),'1')\
                                .when((f.trim(f.col('cd_prod_crra')) == 'C10') & (f.trim(f.col('cd_subp_crra')) == 'CRV') & (~(f.trim(f.col('ds_movi_prod_crra')).like('PROVENTOS%'))),'1')\
                                .otherwise(f.lit('0')).alias('fl_prod'),\
                               f.lit('0').alias('fl_fundo_auto'),\
                               f.lit('0').alias('fl_fundo_espelho'),\
                               f.lit('0').alias('fl_portabilidade'),\
                               f.regexp_replace(f.col('cd_prod_crra'),' ','').alias('cd_prod'),\
                               f.regexp_replace(f.col('cd_subp_crra'),' ','').alias('cd_subp'),\
                               f.lit('NA').alias('cd_subp_tc'),\
                               f.col('dat_ref_carga').alias('dat_ref_carga'),\
                               f.col('dt_liqu_crra').alias('dt_cver_movi'),\
                               f.trim(f.col('tp_moti_prod_crra')).alias('cd_tipo_movi'),\
                               f.trim(f.col('ds_movi_prod_crra')).alias('ds_tipo_movi'),\
                               f.when(f.regexp_replace(f.col('tp_movi_crra'),' ','') == 'E','1')\
                                .when(f.regexp_replace(f.col('tp_movi_crra'),' ','') == 'S','-1').alias('tp_movi'),\
                               f.trim(f.col('cd_cnal_oper')).alias('cd_canal'),\
                               f.trim(f.col('ds_cnal_crra')).alias('ds_canal'),\
                               f.trim(f.col('cd_usua_crra')).alias('cd_ang'),\
                               f.lit(' ').alias('ds_mod_atend'),\
                               f.when(f.regexp_replace(f.col('tp_movi_crra'),' ','') == 'E', abs(f.col('vl_oper_crra').cast('double')))\
                                .when(f.regexp_replace(f.col('tp_movi_crra'),' ','') == 'S', abs(f.col('vl_oper_crra').cast('double'))* (-1)).alias('vl_movi'),\
                               f.when(f.regexp_replace(f.col('cd_prod_crra'),' ','') == 'CRV' , '1')\
                                .otherwise('0').alias('fl_diversificacao'),\
                               f.concat(f.col('dt_liqu_crra').substr(1,4),\
                                        f.col('dt_liqu_crra').substr(6,2)).alias('dt_anomes'),\
                               f.lit('CORRETORA').alias('ds_familia_carga'),\
                               f.regexp_replace(f.concat(f.coalesce(f.col('cd_banc_crra'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_agen_crra'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('nr_cnta_crrt_crra'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_cntr_crra'), f.lit('9999')), f.lit('#'),\
                                                         f.coalesce(f.col('cd_unic_info_crra'), f.lit('9999'))),' ','').alias('pk_key'),\
                               f.lit('EY').alias('sistema_origem'),\
                               f.col('tb_origem').alias('tb_origem')\
                              )

# Filtrando contrato: -32 Bi TERMO.
if dt_anomes_aaaamm == '202302' :
    do211_prod = do211_prod.filter(f.col('pk_key')!='33#1141#10179003#220775207#0000287739FCASWOF20230206FLX0000287739000')
else:
    do211_prod

###########################################################
##############       Bases de Produtos       ##############
##############           POUPANCA            ##############
###########################################################

do201 = spark.read.table('do.dodt201')\
             .filter((f.col('dt_movi') <= dt_prodm1) &\
                     (f.col('dt_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_movi') == 0)\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt201'))

do201_vrj = spark.read.table('do.dodt201_vrj')\
             .filter((f.col('dt_movi') <= dt_prodm1) &\
                     (f.col('dt_movi').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('vl_movi') == 0)\
                    )\
             .withColumn('tb_origem',f.lit('do.dodt201_vrj')) 

##################################################
#### PERIODO DE FECHAMENTO - PEGANDO M0 E M-1 ####
##################################################

if (int(dia_util) > 1) and (int(dia_util) <= 3):
    do201_m0 = spark.read.table('do.dodt201')\
                 .filter((f.col('dt_movi') <= dt_prod_m0) &\
                         (f.col('dt_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_movi') == 0)\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt201'))

    do201_vrj_m0 = spark.read.table('do.dodt201_vrj')\
                 .filter((f.col('dt_movi') <= dt_prod_m0) &\
                         (f.col('dt_movi').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                         ~(f.col('vl_movi') == 0)\
                        )\
                 .withColumn('tb_origem',f.lit('do.dodt201_vrj'))

    do201_full = do201.union(do201_vrj)\
                        .union(do201_m0)\
                        .union(do201_vrj_m0)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc_cnta_poup','cd_agen_cnta_poup','nr_cnta_poup',\
                                                                                 'nr_scnt','nr_sequ_movi_poup','dt_movi')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)
else:
    do201_full = do201.union(do201_vrj)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc_cnta_poup','cd_agen_cnta_poup','nr_cnta_poup',\
                                                                                 'nr_scnt','nr_sequ_movi_poup','dt_movi')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

do201_prod = do201_full.select(f.col('dt_movi').alias('dt_movi'),\
                               f.col('cd_clie').alias('nr_pess'),\
                               f.col('cd_banc_cnta_poup').alias('cd_banc'),\
                               f.col('cd_agen_cnta_poup').alias('cd_agen'),\
                               f.col('nr_cnta_poup').alias('nr_cnta_crrt'),\
                               f.lit(' ').alias('cd_cntr'),\
                               f.lit('0').alias('tx_clie'),\
                               f.lit(' ').alias('tx_tti'),\
                               f.lit(' ').alias('tx_spread'),\
                               f.lit(' ').alias('ds_indx'),\
                               f.col('nr_sequ_movi_poup').alias('nr_sequ_movi'),\
                               f.lit(' ').alias('cd_fundo'),\
                               f.lit(' ').alias('ds_fundo'),\
                               f.lit('9999-99-99').alias('dt_vencimento'),\
                              f.when((f.trim(f.col('ds_tipo_movi')).isin('JUROS','RESGATE CONTAMAX','REMUNERACAO CONTAMAX CDB DI')),'0')\
                               .when((f.trim(f.col('ds_tipo_movi')).like('%REMUNERACAO%')),'0')\
                               .when((f.trim(f.col('ds_tipo_movi')).like('%JUROS%')),'0')\
                                 .otherwise('1').alias('fl_prod'),\
                               f.lit('0').alias('fl_fundo_auto'),\
                               f.lit('0').alias('fl_fundo_espelho'),\
                               f.lit('0').alias('fl_portabilidade'),\
                               f.lit('51').alias('cd_prod'),\
                               f.col('cd_subp_poup').alias('cd_subp'),\
                               f.lit('NA').alias('cd_subp_tc'),\
                               f.col('dat_ref_carga').alias('dat_ref_carga'),\
                               f.col('dt_movi').alias('dt_cver_movi'),\
                               f.trim(f.col('tp_movi')).alias('cd_tipo_movi'),\
                               f.col('ds_tipo_movi').alias('ds_tipo_movi'),\
                               f.when(((f.trim(f.col('tp_movi_inve')) == 'E') & ((f.upper(f.col('ds_tipo_movi')).like('%ESTOR%')) | \
                                                                                (f.upper(f.col('ds_tipo_movi')).like('%DEVOL%')))),'-1')\
                                .when(((f.trim(f.col('tp_movi_inve')) == 'S') & ((f.upper(f.col('ds_tipo_movi')).like('%ESTOR%'))  | \
                                                                                (f.upper(f.col('ds_tipo_movi')).like('%DEVOL%')))),'1')\
                                .when(((f.trim(f.col('tp_movi_inve')) == 'E') & (~(f.upper(f.col('ds_tipo_movi')).like('%ESTOR%')) & \
                                                                                ~(f.upper(f.col('ds_tipo_movi')).like('%DEVOL%')))),'1')\
                                .when(((f.trim(f.col('tp_movi_inve')) == 'S') & (~(f.upper(f.col('ds_tipo_movi')).like('%ESTOR%')) & \
                                                                                ~(f.upper(f.col('ds_tipo_movi')).like('%DEVOL%')))),'-1')\
                                .alias('tp_movi'),\
                               f.when(f.trim(f.col('tp_movi_inve')) == 'E', abs(f.col('vl_movi')))\
                                .when(f.trim(f.col('tp_movi_inve')) == 'S', abs(f.col('vl_movi'))*(-1)).alias('vl_movi'),\
                               f.trim(f.col('cd_cnal_oper')).alias('cd_canal'),\
                               f.lit(' ').alias('ds_canal'),\
                               f.trim(f.col('cd_usua_oper')).alias('cd_ang'),\
                               f.lit(' ').alias('ds_mod_atend'),\
                               f.lit('0').alias('fl_diversificacao'),\
                               f.concat(f.col('dt_movi').substr(1,4),\
                                        f.col('dt_movi').substr(6,2)).alias('dt_anomes'),\
                               f.lit('POUPANCA').alias('ds_familia_carga'),\
                               f.concat(f.coalesce(f.col('cd_banc_cnta_poup'), f.lit('9999')), f.lit('#'),\
                                        f.coalesce(f.col('cd_agen_cnta_poup'), f.lit('9999')), f.lit('#'),\
                                        f.coalesce(f.col('nr_cnta_poup'), f.lit('9999'))).alias('pk_key'),\
                               f.lit('BG').alias('sistema_origem'),\
                               f.col('tb_origem').alias('tb_origem')\
                              )


###########################################################
##############       Bases de Produtos       ##############
##############           NOVO COE            ##############
###########################################################

do215CN = spark.read.table('do.dodt215')\
                    .filter((f.col('dt_oper_sigom') <= dt_prodm1) &\
                            (f.col('dt_oper_sigom').substr(1,7) == dt_anomes_aaaa_mm) &\
                            (f.col('sg_sist') == 'NW') &\
                            ~(f.col('vl_apli_sigom') == 0)\
                           )\
                    .withColumn('tb_origem',f.lit('do.dodt215'))

##################################################
#### PERIODO DE FECHAMENTO - PEGANDO M0 E M-1 ####
##################################################

if (int(dia_util) > 1) and (int(dia_util) <= 3):
    
    do215CN_m0 = spark.read.table('do.dodt215')\
                        .filter((f.col('dt_oper_sigom') <= dt_prod_m0) &\
                                (f.col('dt_oper_sigom').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                                (f.col('sg_sist') == 'NW') &\
                                ~(f.col('vl_apli_sigom') == 0)
                               )\
                        .withColumn('tb_origem',f.lit('do.dodt215'))
    
    do215CN_full = do215CN.union(do215CN_m0)\
                            .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt','cd_unic_info_sigom','dt_oper_sigom', 'nr_sequ_movi_sigom')\
                                                           .orderBy(f.col('dat_ref_carga').desc())))\
                            .filter(f.col('rank') == 1)

else:
    do215CN_full = do215CN.withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt','cd_unic_info_sigom','dt_oper_sigom', 'nr_sequ_movi_sigom')\
                                                           .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)
    
do215CN_prod = do215CN_full.select(f.col('dt_movi_sigom').alias('dt_movi'),\
                                   f.col('cd_clie').alias('nr_pess'),\
                                   f.col('cd_banc').alias('cd_banc'),\
                                   f.col('cd_agen').alias('cd_agen'),\
                                   f.col('nr_cnta_crrt').alias('nr_cnta_crrt'),\
                                   f.col('cd_unic_info_sigom').alias('cd_cntr'),\
                                   f.col('pc_indx_ativ_sigom').alias('tx_clie'),\
                                   f.lit(' ').alias('tx_tti'),\
                                   f.lit(' ').alias('tx_spread'),\
                                   f.regexp_replace(f.col('nm_indx_sigom'),' ','').alias('ds_indx'),\
                                   f.col('nr_sequ_movi_sigom').alias('nr_sequ_movi'),\
                                   f.lit(' ').alias('cd_fundo'),\
                                   f.lit(' ').alias('ds_fundo'),\
                                   f.col('dt_venc_oper_sigom').alias('dt_vencimento'),\
                                   f.when(((f.col('tp_movi_sigom') == 13) & (f.trim(f.col('tp_movi_inve_sigom')) == 'S')),'0')\
                                    .when(f.trim(f.col('cd_subp_teso')) == '0220','1')\
                                    .otherwise('1').alias('fl_prod'),\
                                   f.lit('0').alias('fl_fundo_auto'),\
                                   f.lit('0').alias('fl_fundo_espelho'),\
                                   f.lit('0').alias('fl_portabilidade'),\
                                   f.col('cd_prod_sigom').alias('cd_prod'),\
                                   f.col('cd_subp_sigom').alias('cd_subp'),\
                                   f.col('cd_subp_teso').alias('cd_subp_tc'),\
                                   f.col('dat_ref_carga').alias('dat_ref_carga'),\
                                   f.col('dt_oper_sigom').alias('dt_cver_movi'),\
                                   f.col('tp_movi_sigom').alias('cd_tipo_movi'),\
                                   f.col('nm_tipo_movi_sigom').alias('ds_tipo_movi'),\
                                   f.when(f.trim(f.col('tp_movi_inve_sigom')) == 'E', '1')\
                                    .when(f.trim(f.col('tp_movi_inve_sigom')) == 'S', '-1').alias('tp_movi'),\
                                   f.when((f.trim(f.col('tp_movi_inve_sigom')) == 'E') &\
                                          (f.trim(f.col('nm_tipo_movi_sigom')) == 'CANCELAMENTO'), abs(f.col('vl_apli_sigom')))\
                                    .when(f.trim(f.col('tp_movi_inve_sigom')) == 'E', abs(f.col('vl_brut_oper_sigom')))\
                                    .when((f.trim(f.col('tp_movi_inve_sigom')) == 'S') &\
                                          (f.trim(f.col('nm_tipo_movi_sigom')) == 'CANCELAMENTO'), abs(f.col('vl_apli_sigom'))*(-1))\
                                    .when(f.trim(f.col('tp_movi_inve_sigom')) == 'S', abs(f.col('vl_brut_oper_sigom'))*(-1)).alias('vl_movi'),\
                                   f.col('cd_cnal_oper').alias('cd_canal'),\
                                   f.lit(' ').alias('ds_canal'),\
                                   f.col('cd_usua_oper').alias('cd_ang'),\
                                   f.lit(' ').alias('ds_mod_atend'),\
                                   f.lit('1').alias('fl_diversificacao'),\
                                   f.concat(f.col('dt_oper_sigom').substr(1,4),\
                                            f.col('dt_oper_sigom').substr(6,2)).alias('dt_anomes'),\
                                   f.lit('NOVO COE').alias('ds_familia_carga'),\
                                   f.concat(f.coalesce(f.col('cd_banc'), f.lit('9999')), f.lit('#'),\
                                            f.coalesce(f.col('cd_agen'), f.lit('9999')), f.lit('#'),\
                                            f.coalesce(f.col('nr_cnta_crrt'), f.lit('9999')), f.lit('#'),\
                                            f.coalesce(f.col('cd_unic_info_sigom'), f.lit('9999')), f.lit('#'),\
                                            f.coalesce(f.col('nr_sequ_movi_sigom'), f.lit('9999'))).alias('pk_key'),\
                                   f.lit('NW').alias('sistema_origem'),\
                                   f.col('tb_origem').alias('tb_origem')\
                                  )

###########################################################
##############       Bases de Produtos       ##############
#######           CRI/CRA/DEB/TITULOSPUB            #######
###########################################################

do215CCD = spark.read.table('do.dodt215')\
             .filter((f.col('dt_oper_sigom') <= dt_prodm1) &\
                     (f.col('dt_oper_sigom').substr(1,7) == dt_anomes_aaaa_mm) &\
                     ~(f.col('sg_sist') == 'NW') &\
                     ~(f.col('vl_apli_sigom') == 0)
                    )\
             .withColumn('tb_origem',f.lit('do.dodt215'))

##################################################
#### PERIODO DE FECHAMENTO - PEGANDO M0 E M-1 ####
##################################################

if (int(dia_util) > 1) and (int(dia_util) <= 3):
    do215CCD_m0 = spark.read.table('do.dodt215')\
                        .filter((f.col('dt_oper_sigom') <= dt_prod_m0) &\
                                (f.col('dt_oper_sigom').substr(1,7) == dt_anomes_aaaa_mm_m0) &\
                                ~(f.col('sg_sist') == 'NW') &\
                                ~(f.col('vl_apli_sigom') == 0)
                               )\
                        .withColumn('tb_origem',f.lit('do.dodt215'))

    do215CCD_full = do215CCD.union(do215CCD_m0)\
                        .withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt','cd_unic_info_sigom','dt_oper_sigom', 'nr_sequ_movi_sigom')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)
    
else:
    do215CCD_full = do215CCD.withColumn('rank', f.row_number().over(Window.partitionBy('cd_banc','cd_agen','nr_cnta_crrt','cd_unic_info_sigom','dt_oper_sigom', 'nr_sequ_movi_sigom')\
                                                                    .orderBy(f.col('dat_ref_carga').desc())))\
                        .filter(f.col('rank') == 1)

do215CCD_prod = do215CCD_full.select(f.col('dt_movi_sigom').alias('dt_movi'),\
                                       f.col('cd_clie').alias('nr_pess'),\
                                       f.col('cd_banc').alias('cd_banc'),\
                                       f.col('cd_agen').alias('cd_agen'),\
                                       f.col('nr_cnta_crrt').alias('nr_cnta_crrt'),\
                                       f.col('cd_unic_info_sigom').alias('cd_cntr'),\
                                       f.col('pc_indx_ativ_sigom').alias('tx_clie'),\
                                       f.lit(' ').alias('tx_tti'),\
                                       f.lit(' ').alias('tx_spread'),\
                                       f.col('nm_indx_sigom').alias('ds_indx'),\
                                       f.col('nr_sequ_movi_sigom').alias('nr_sequ_movi'),\
                                       f.lit(' ').alias('cd_fundo'),\
                                       f.lit(' ').alias('ds_fundo'),\
                                       f.col('dt_venc_oper_sigom').alias('dt_vencimento'),\
                                       f.when((f.trim(f.col('cd_subp_sigom')).isin('9500','1550','5','100','101','200','201', 'LIG IPCA')) &\
                                              (f.trim(f.col('tp_movi_sigom')).cast('int').isin(1,4,9,10)),'1')\
                                        .otherwise('0').alias('fl_prod'),\
                                       f.lit('0').alias('fl_fundo_auto'),\
                                       f.lit('0').alias('fl_fundo_espelho'),\
                                       f.lit('0').alias('fl_portabilidade'),\
                                       f.col('cd_prod_sigom').alias('cd_prod'),\
                                       f.col('cd_subp_sigom').alias('cd_subp'),\
                                       f.col('cd_ativ_nego_sigom').alias('cd_subp_tc'),\
                                       f.col('dat_ref_carga').alias('dat_ref_carga'),\
                                       f.col('dt_oper_sigom').alias('dt_cver_movi'),\
                                       f.col('tp_movi_sigom').alias('cd_tipo_movi'),\
                                       f.col('nm_tipo_movi_sigom').alias('ds_tipo_movi'),\
                                       f.when(f.trim(f.col('tp_movi_inve_sigom')) == 'E', '1')\
                                        .when(f.trim(f.col('tp_movi_inve_sigom')) == 'S', '-1').alias('tp_movi'),\
                                       f.when(f.trim(f.col('tp_movi_inve_sigom')) == 'E', abs(f.col('vl_apli_sigom')))\
                                        .when(f.trim(f.col('tp_movi_inve_sigom')) == 'S', abs(f.col('vl_apli_sigom'))*(-1)).alias('vl_movi'),\
                                       f.col('cd_cnal_oper').alias('cd_canal'),\
                                       f.lit(' ').alias('ds_canal'),\
                                       f.col('cd_usua_oper').alias('cd_ang'),\
                                       f.lit(' ').alias('ds_mod_atend'),\
                                       f.lit('1').alias('fl_diversificacao'),\
                                       f.concat(f.col('dt_oper_sigom').substr(1,4),\
                                                f.col('dt_oper_sigom').substr(6,2)).alias('dt_anomes'),\
                                       f.lit('CRI/CRA/DEBENTURE/TPUBTESO').alias('ds_familia_carga'),\
                                       f.concat(f.coalesce(f.col('cd_banc'), f.lit('9999')), f.lit('#'),\
                                                f.coalesce(f.col('cd_agen'), f.lit('9999')), f.lit('#'),\
                                                f.coalesce(f.col('nr_cnta_crrt'), f.lit('9999')), f.lit('#'),\
                                                f.coalesce(f.col('cd_unic_info_sigom'), f.lit('9999')), f.lit('#'),\
                                                f.coalesce(f.col('nr_sequ_movi_sigom'), f.lit('9999'))).alias('pk_key'),\
                                       f.lit('SBK').alias('sistema_origem'),\
                                       f.col('tb_origem').alias('tb_origem')\
                                      )

In [ ]:
###########################################################
##############       Bases de Produtos       ##############
##############           BASE FINAL          ##############
###########################################################

prod_tot = do215CN_prod.unionByName(do215CCD_prod)\
                       .unionByName(do201_prod)\
                       .unionByName(do211_prod)\
                       .unionByName(do205_prod)\
                       .unionByName(do203_prod)\
                       .unionByName(do207_prod)

prodJoin_pe = prod_tot.join(pe001_seg, f.lpad(prod_tot['nr_pess'],8,'0') == f.lpad(pe001_seg['penumper'],8,'0'),'left')\
                         .select(prod_tot['*'],\
                                 pe001_seg['penumero'].alias('nr_cpf_cnpj'),\
                                 pe001_seg['penomper'].alias('nm_clie'),\
                                 pe001_seg['pesubseg'].alias('cd_segmento'),\
                                 pe001_seg['nome_segmento'].alias('ds_segmento'),\
                                 pe001_seg['sub_segmento'].alias('ds_segmento_agr')\
                                )\
                         .withColumn('cd_subp_oq', f.when(f.col('ds_familia_carga') == 'PREVIDENCIA', f.col('cd_fundo'))\
                                                    .when(f.col('ds_familia_carga').isin('CRI/CRA/DEBENTURE/TPUBTESO','NOVO COE'), f.col('cd_subp_tc'))\
                                                    .otherwise(f.col('cd_subp')))\
                         .withColumn('cd_subp_div', f.when(f.col('ds_familia_carga').isin('PREVIDENCIA','FUNDOS','NOVO COE'), f.col('cd_subp_tc'))\
                                                    .otherwise(f.col('cd_subp')))

In [ ]:
prodJoin = prodJoin_pe.join(do003, (prodJoin_pe['cd_prod'] == do003['cd_prod']) & \
                                   (f.lpad(prodJoin_pe['cd_subp'],10,'0') == f.lpad(do003['cd_subp'],10,'0')) ,'left')\
                      .join(tbCli, f.lpad(prodJoin_pe['nr_pess'],8,'0') == f.lpad(tbCli['penumper'],8,'0'),'left')\
                      .join(tbOq, (prodJoin_pe['sistema_origem'] == tbOq['sistema_origem']) & \
                                  (prodJoin_pe['cd_prod'] == tbOq['cd_prod']) & \
                                  (f.lpad(prodJoin_pe['cd_subp_oq'],20,'0') == f.lpad(tbOq['cd_subp'],20,'0')), 'left')\
                      .join(tbProdDiv, (prodJoin_pe['sistema_origem'] == tbProdDiv['sistema_origem']) & \
                                       (prodJoin_pe['cd_prod'] == tbProdDiv['cd_prod']) & \
                                       (f.lpad(prodJoin_pe['cd_subp_div'],20,'0') == f.lpad(tbProdDiv['cd_subp'],20,'0')), 'left')\
                      .join(tbVenc, (f.lpad(prodJoin_pe['nr_pess'],8,'0') == f.lpad(tbVenc['cd_clie'],8,'0')) & \
                                    (prodJoin_pe['cd_cntr'] == tbVenc['cd_cntr']),'left')\
                      .join(cnal, f.lpad(cnal['cd_canal'],4,'0') == f.lpad(prodJoin_pe['cd_canal'],4,'0'),'left')\
                      .select(prodJoin_pe['dt_movi'],\
                              prodJoin_pe['nr_pess'],\
                              prodJoin_pe['nr_cpf_cnpj'],\
                              tbCli['nm_clie'].alias('nm_clie'),\
                              prodJoin_pe['cd_segmento'],\
                              prodJoin_pe['ds_segmento'],\
                              prodJoin_pe['ds_segmento_agr'],\
                              tbCli['ds_perfil_api'],\
                              f.coalesce(tbCli['clie_assessoria'],f.lit('0')).alias('clie_assessoria'),\
                              tbCli['nm_coord'],\
                              tbCli['matr_coord'],\
                              tbCli['nm_assessor'],\
                              tbCli['matr_assessor'],\
                              tbCli['modelo_atnd'],\
                              tbCli['nm_pais_assessor'],\
                              f.coalesce(tbCli['fl_sem_assessor'],f.lit('0')).alias('fl_sem_assessor'),\
                              prodJoin_pe['cd_banc'],\
                              prodJoin_pe['cd_agen'],\
                              prodJoin_pe['nr_cnta_crrt'],\
                              tbCli['uniorg'],\
                              tbCli['rede'],\
                              tbCli['regional'],\
                              tbCli['pv'],\
                              prodJoin_pe['cd_cntr'],\
                              prodJoin_pe['tx_clie'],\
                              prodJoin_pe['tx_tti'],\
                              prodJoin_pe['tx_spread'],\
                              prodJoin_pe['ds_indx'],\
                              prodJoin_pe['nr_sequ_movi'],\
                              prodJoin_pe['cd_fundo'],\
                              prodJoin_pe['ds_fundo'],\
                              f.when(prodJoin_pe['ds_familia_carga'] == 'RENDA FIXA',tbVenc['dt_venc_oper'])\
                               .otherwise(prodJoin_pe['dt_vencimento']).alias('dt_vencimento'),\
                              f.when(prodJoin_pe['nr_pess'] == '28195667', f.lit('0'))\
                                .otherwise(prodJoin_pe['fl_prod']).alias('fl_prod'),\
                              prodJoin_pe['fl_fundo_auto'],\
                              prodJoin_pe['fl_portabilidade'],\
                              prodJoin_pe['cd_prod'],\
                              f.when(prodJoin_pe['ds_familia_carga'] == 'RENDA FIXA',f.regexp_replace(do003['nm_subp'].substr(1,3),' ',''))\
                               .when((prodJoin_pe['ds_familia_carga'] == 'CORRETORA') & (prodJoin_pe['cd_subp'] == 'CRV'), 'ALGO+')\
                               .otherwise(do003['nm_prod']).alias('ds_produto'),\
                              prodJoin_pe['cd_subp'],\
                              f.when((prodJoin_pe['ds_familia_carga'] == 'CORRETORA') & (prodJoin_pe['cd_subp'] == 'CRV'),prodJoin_pe['ds_tipo_movi'])\
                               .otherwise(do003['nm_subp']).alias('ds_subp'),\
                              f.when((prodJoin_pe['ds_familia_carga'] == 'FUNDOS'),do003['CD_SUBP_TC'])\
                               .otherwise(prodJoin_pe['cd_subp_tc']).alias('cd_subp_tc'),\
                              f.when(prodJoin_pe['ds_familia_carga'] == 'CRI/CRA/DEBENTURE/TPUBTESO', 'CREDITO PRIVADO')\
                               .otherwise(tbOq['classe_ativo']).alias('ds_classe_ativo'),\
                              prodJoin_pe['dat_ref_carga'],\
                              prodJoin_pe['dt_cver_movi'],\
                              prodJoin_pe['cd_tipo_movi'],\
                              prodJoin_pe['ds_tipo_movi'],\
                              prodJoin_pe['tp_movi'],\
                              prodJoin_pe['vl_movi'],\
                              prodJoin_pe['cd_canal'],\
                              f.when(prodJoin_pe['ds_familia_carga'] == 'CORRETORA', prodJoin_pe['ds_canal'])
                                .otherwise(cnal['ds_canal']).alias('ds_canal'),\
                              prodJoin_pe['cd_ang'],\
                              prodJoin_pe['ds_mod_atend'],\
                              f.when(tbProdDiv['fl_fundo_espelho'] == '1', '1')\
                               .otherwise('0').alias('fl_fundo_espelho'),\
                              f.when(tbProdDiv['fl_diversificacao'] == '1','1')\
                               .otherwise(prodJoin_pe['fl_diversificacao']).alias('fl_diversificacao'),\
                              prodJoin_pe['pk_key'],\
                              prodJoin_pe['sistema_origem'],\
                              prodJoin_pe['tb_origem'],\
                              prodJoin_pe['ds_familia_carga'],\
                              prodJoin_pe['dt_anomes']\
                              )

if dt_anomes_aaaa_mm == '2022-12' :
    lf = spark.read.table('sand_invest.tb_inclusao_oper_lf')
    prodJoin = prodJoin.unionByName(lf)
else:
    prodJoin

In [ ]:
if (int(dia_util) > 1) and (int(dia_util) <= 3):
    print('Fluxo de fechamento:')
    print('Dropando a particao do fechamento!')
    spark.sql("ALTER TABLE sand_invest.tb_prod_lake_new DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_anomes_aaaamm)) 
    print('Dropando a particao do mês corrente!')
    spark.sql("ALTER TABLE sand_invest.tb_prod_lake_new DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_anomes_aaaamm_m0)) 
else:
    print('Fluxo normal, dropando partição do mês atual:')
    spark.sql("ALTER TABLE sand_invest.tb_prod_lake_new DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_anomes_aaaamm)) 

In [ ]:
prodJoin.write.mode("append").partitionBy('dt_anomes','ds_familia_carga').saveAsTable('sand_invest.tb_prod_lake_new')

In [ ]:
###########################################################
############   Verifica data de carga origem   ############
###########################################################

tb_origem = ['do.dodt201','do.dodt201_vrj','do.dodt203','do.dodt203_vrj','do.dodt205','do.dodt205_vrj','do.dodt207','do.dodt207_vrj','do.dodt211','do.dodt211_vrj', 'do.dodt215']

tb_origem_d1 = ['do.dodt201','do.dodt201_vrj','do.dodt203','do.dodt203_vrj','do.dodt205','do.dodt205_vrj','do.dodt207','do.dodt207_vrj','do.dodt211','do.dodt211_vrj']

if int(dia_util) <= 3:
    first = datetime.today().replace(day=1) # Reseta o dia da data atual para o dia 1
    res = first - timedelta(days=1) # Pega o dia anterior ao primeiro (ultimo dia do mes anterior)
    dt_comparacao_1 = wd.workdays(res.date(), -1, country='BR')
    dt_comparacao_2 = wd.workdays(res.date(), -2, country='BR')
    
    df_dq_prod_ini = spark.read.table('sand_invest.tb_prod_lake_new')\
                            .filter((f.col('dt_anomes')==str(dt_comparacao_1.strftime('%Y%m')))&\
                                    (f.col('tb_origem').isin(tb_origem))&\
                                    (f.col('fl_prod')=='1'))\
                            .select('tb_origem', 'dt_cver_movi')\
                            .groupby('tb_origem')\
                                .agg(f.max('dt_cver_movi').alias('max_dt_cver_movi'))\
                            .withColumn('dat_ref_carga', f.lit(str(dt_today)))

    df_dq_prod = df_dq_prod_ini.withColumn("verifica_data_carga", 
                                    f.when((f.col('tb_origem').isin(tb_origem_d1))&(f.col('max_dt_cver_movi') >= str(dt_comparacao_1)),"OK")\
                                     .when((f.col('tb_origem')=='do.dodt215')&(f.col('max_dt_cver_movi') >= str(dt_comparacao_2)),"OK")\
                                     .otherwise(f.lit('NOK')))

    df_dq_prod.write.mode("overwrite").saveAsTable('sand_invest.tb_dq_prod_tb_origem')
    verifica_prod = df_dq_prod.filter(f.col('verifica_data_carga')=='NOK')
    qtd_nok = verifica_prod.count()
    if qtd_nok >= 1:
        status_carga = 'Origem sem carga'
        print(f'Status: {status_carga}\nQtd NOK: {qtd_nok}')
    else:
        status_carga = 'Concluido'
        print(f'Status: {status_carga}\nQtd NOK: {qtd_nok}')

else:
    dt_comparacao_1 = wd.workdays(dt_today, -1, country='BR')
    dt_comparacao_2 = wd.workdays(dt_today, -2, country='BR')
                                                 
    df_dq_prod_ini = spark.read.table('sand_invest.tb_prod_lake_new')\
                            .filter((f.col('dt_anomes')==str(dt_comparacao_1.strftime('%Y%m')))&\
                                    (f.col('tb_origem').isin(tb_origem))&\
                                    (f.col('fl_prod')=='1'))\
                            .select('tb_origem', 'dt_cver_movi')\
                            .groupby('tb_origem')\
                                .agg(f.max('dt_cver_movi').alias('max_dt_cver_movi'))\
                            .withColumn('dat_ref_carga', f.lit(str(dt_today)))

    df_dq_prod = df_dq_prod_ini.withColumn("verifica_data_carga", 
                                    f.when((f.col('tb_origem').isin(tb_origem_d1))&(f.col('max_dt_cver_movi') >= str(dt_comparacao_1)),"OK")\
                                     .when((f.col('tb_origem')=='do.dodt215')&(f.col('max_dt_cver_movi') >= str(dt_comparacao_2)),"OK")\
                                     .otherwise(f.lit('NOK')))

    df_dq_prod.write.mode("overwrite").saveAsTable('sand_invest.tb_dq_prod_tb_origem')
    verifica_prod = df_dq_prod.filter(f.col('verifica_data_carga')=='NOK')
    qtd_nok = verifica_prod.count()
    if qtd_nok >= 1:
        status_carga = 'Origem sem carga'
        print(f'Status: {status_carga}\nQtd NOK: {qtd_nok}')
    else:
        status_carga = 'Concluido'
        print(f'Status: {status_carga}\nQtd NOK: {qtd_nok}')

In [ ]:
###########################################################
################       Grava Tabelas       ################
################    DQ - Step Controle     ################
###########################################################
################         - Fim -           ################
###########################################################
    
dt_exec_fim = datetime.today()
  
dq_wf_fim = spark.read.table('sand_invest.tb_dq_qlik_wf')\
                 .filter((f.col('ds_processo') == 'prod') &\
                         (f.col('status_carga') == 'Iniciado') &\
                         (f.col('dat_ref_carga') == dt_today))\
                 .withColumn('fim_time_carga', f.lit(str(dt_exec_fim)))\
                 .select(f.col('ds_processo'),\
                         f.lit(status_carga).alias('status_carga'),\
                         f.col('fim_time_carga').alias('time_carga'),\
                         ((f.to_timestamp(f.col('fim_time_carga')).cast('long') - \
                           f.to_timestamp(f.col('time_carga')).cast('long')) / 60).alias('delta_time_carga'),\
                         f.lit('0').alias('qtd_reg_dup'),\
                         f.lit(dt_prodm1).alias('dt_posi'),\
                         f.lit(dt_anomes_aaaamm).alias('dt_anomes'),\
                         f.lit(dt_today).alias('dat_ref_carga'))
    
dq_wf_fim.write.mode("append").partitionBy('ds_processo','dt_anomes','dat_ref_carga').saveAsTable('sand_invest.tb_dq_qlik_wf')

#### Gerando o dq da prod

In [ ]:
try:
    df_dq_prod_pd = df_dq_prod.toPandas()
    dash_dq_prod('t786430', 'FE@290198', 'N2501422', 'bsbrsp54','bs.br.bsch', df_dq_prod_pd)
except Exception as e:
    print(e)

#### Gerando painel de acompanhamento de processos na rede

In [ ]:
try:
    df_dq_wf = spark.read.table("sand_invest.tb_dq_qlik_wf")\
                    .filter((f.col("dt_anomes")==f"{dt_anomes_aaaamm}"))\
                    .withColumn("ds_area", f.when((f.col('ds_processo').isin("cli", "cash", "posi", "prod")), f.lit("MI")).otherwise(f.lit("BI")))\
                    .orderBy(f.col("ds_processo").asc(), f.col("status_carga").asc())
    df_dq_wf_pd = df_dq_wf.toPandas().sort_values(by=['time_carga', 'ds_processo'],ascending=True)
    df_dq_wf_pd.time_carga = pd.to_datetime(df_dq_wf_pd.time_carga).astype(str)
    painel_resumo_processos('t786430', 'FE@290198', 'N2501422', 'bsbrsp54','bs.br.bsch', 'prod', df_dq_wf_pd)
except Exception as e:
    print(e)

<br>

## INICIO PULSO

In [ ]:
###  PRODUCAO ANUAL
prod = spark.read.table('sand_invest.tb_prod_lake_new')\
            .filter((f.col('dt_anomes') == dt_anomes_aaaamm) &\
                    ((f.col('fl_prod') == '1') | (f.col('ds_produto') == 'FUNDOS'))\
                   )

In [ ]:
 ### Anbima
anb = spark.read.table('sand_invest.tb_class_anbima')\
           .filter(~(f.col('cd_mx') == '0'))

In [ ]:
 ### dia útil
du = spark.read.table('sand_invest.tb_du')\
          .filter(f.col('dt_anomes') == dt_anomes_aaaamm)\

In [ ]:
 ### Clientes
cli = spark.read.table('sand_invest.tb_clientes_lake_new')\
           .filter(f.col('dt_anomes') == dt_anomes_aaaamm)

In [ ]:
#estr = spark.read.table('sand_invest.tb_bis_estrutura')\
#            .filter((f.col('dt_anomes') == dt_anomes_aaaamm) &\
#                    (f.col('dat_ref_carga') == dt_estr))\
#            .withColumn('clie_assessoria',f.lit('1'))\

estr = spark.read.table('sand_invest.tb_bis_estrutura')\
            .filter((f.col('dat_ref_carga') == dt_estr))\
            .withColumn('clie_assessoria',f.lit('1'))\

### PULSO HIST

In [ ]:
### PULSO HIST
pulso_hist = prod.join(cli, prod['nr_pess'] == cli['penumper'],'left')\
                 .join(estr, f.lpad(prod['nr_pess'],8,'0') == f.lpad(estr['penumper'],8,'0'), 'left')\
                 .join(anb, prod['cd_fundo'].cast('float') == anb['cd_mx'].cast('float'),'left')\
                 .join(du, prod['dt_cver_movi'] == du['data'],'left')\
                 .select(prod['dt_anomes'].alias('dt_anomes_hist'),\
                         prod['dt_cver_movi'].alias('dt_cver_movi_hist'),\
                         f.when(prod['ds_produto'].isin('ACOES','COMMODITIES','DIREITO E BONUS','ETF','TITULOS PRIVADOS','SWAP'),'PRODUTOS BOLSA')\
                          .when(prod['ds_produto'] == 'ALGO+', 'ALGO +')\
                          .when(((prod['ds_familia_carga'] == 'FUNDOS') & (prod['fl_fundo_auto'] == '1')),'FUNDOS AUTOMATICOS')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & ((f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               (anb['ds_class_anbima'] == '0')),'FUNDOS RF')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & (~(f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               ~(anb['ds_class_anbima'] == '0')),'FUNDOS ESTRUTURADOS')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['cd_subp'].cast('double') == 1550 ), 'CRI')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['cd_subp'].cast('double') == 9500 ), 'CRA')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['cd_subp'].cast('double') == 5 ), 'DEBENTURE')\
                          .when((prod['ds_produto'] == 'PUBLICO'), 'TITULOS PUBLICOS')\
                          .when(prod['ds_familia_carga'] == 'PUBLICO', 'TITULOS PUBLICOS')\
                          .when(prod['ds_familia_carga'] == 'PREVIDENCIA', 'PREVIDENCIA')\
                          .when(prod['ds_produto'] == 'COM', 'COMPROMISSADAS')\
                          .when(prod['cd_subp'] == 'LIG IPCA', 'LIG')\
                          .otherwise(prod['ds_produto']).alias('ds_produto_hist'),\
                         f.when(prod['ds_segmento_agr'].isin('CLASSICO','ESPECIAL'), 'ESPECIAL')\
                          .when(prod['ds_segmento_agr'] == 'VAN_GOGH', 'VAN GOGH')\
                          .when(prod['cd_segmento'].cast('double').isin(15,227),'MEI')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_1', 'EMPRESAS 1')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_2', 'EMPRESAS 2')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_3', 'EMPRESAS 3')\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'), 'GI')\
                           .otherwise(prod['ds_segmento_agr']).alias('ds_segmento_hist'),\
                         f.when((prod['tp_movi'] == '1'),prod['vl_movi'])\
                                     .otherwise('0').alias('aplicacao'),\
                         f.when((prod['tp_movi'] == '-1'),prod['vl_movi'])\
                                      .otherwise('0').alias('resgate'),\
                         prod['vl_movi'],\
                         f.when(prod['ds_segmento_agr'].isin('SELECT','VAN_GOGH','ESPECIAL','CLASSICO'),'PF')\
                          .when(prod['ds_segmento_agr'].isin('EMPRESAS_1','EMPRESAS_2','EMPRESAS_3'),'EMPRESAS')\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES','UNIVERSIDADES'),'GI/UNI').alias('PF_PJ_hist'),\
                         f.when(prod['ds_produto'] == 'COM', 'COMPROMISSADAS')\
                          .when(prod['ds_familia_carga'].isin('RENDA FIXA','NOVO COE'),prod['ds_produto'])\
                          .when(prod['ds_produto'] == 'ALGO+', 'ALGO +')\
                          .when(prod['ds_produto'] == 'TESOURO DIRETO', 'TESOURO DIRETO')\
                          .when(prod['ds_familia_carga'].isin('CORRETORA'),'PRODUTOS BOLSA')\
                          .when(((prod['ds_produto'] == 'FUNDOS') & (prod['fl_fundo_auto'] == '1')),'FUNDOS AUTOMATICOS')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & ((f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               (anb['ds_class_anbima'] == '0')),'FUNDOS RF')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & (~(f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               ~(anb['ds_class_anbima'] == '0')),'FUNDOS ESTRUTURADOS')\
                          .when(prod['ds_produto'].isin('TESOURARIA  SIGOM'),'CREDITO PRIVADO')\
                          .when(prod['ds_produto'] == 'PUBLICO', 'TITULOS PUBLICOS')\
                          .when(prod['cd_subp'] == 'LIG IPCA', 'LIG')\
                          .otherwise(prod['ds_familia_carga']).alias('produto_hist'),\
                         f.when(prod['cd_segmento'].cast('double').isin(15,227),f.lit('0'))\
                          .when(prod['ds_segmento_agr'].isin('EMPRESAS_1','EMPRESAS_2','EMPRESAS_3','GOVERNOS','INSTITUICOES','UNIVERSIDADES'),f.lit('1'))
                          .otherwise(f.coalesce(estr['clie_assessoria'],f.lit('0'))).alias('clie_assessoria_hist'),\
                         f.lit('0').alias('fl_core_hist'),\
                         f.coalesce(f.trim(cli['rede']),f.lit('SEM REDE')).alias('rede_hist'),\
                         f.coalesce(f.trim(cli['regional']),f.lit('SEM REGIONAL')).alias('regional_hist'),\
                         f.coalesce(f.trim(cli['pv']),f.lit('SEM PV')).alias('pv_hist'),\
                         f.when((~(prod['cd_segmento'].cast('double').isin(15,227)) &\
                                (prod['ds_segmento_agr'].like('EMPRESAS%'))),f.coalesce(f.trim(estr['nm_coord']),f.lit('ALEXANDRE NOGUEIRA DE SOUZA')))\
                          .when(prod['ds_segmento_agr'].isin('UNIVERSIDADES'),f.coalesce(f.trim(estr['nm_coord']),f.lit('ALEXANDRE NOGUEIRA DE SOUZA')))\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'),f.coalesce(f.trim(estr['nm_coord']),f.lit('ALEXANDRE LEITZKE DEL PINO')))\
                          .otherwise(estr['nm_coord']).alias('nm_coord_hist'),\
                         f.coalesce(estr['nome_tl'],f.lit('SEM TEAM LEADER')).alias('nm_tl_hist'),\
                         f.when((~(prod['cd_segmento'].cast('double').isin(15,227)) &\
                                (prod['ds_segmento_agr'].like('EMPRESAS%'))),f.coalesce(f.trim(estr['nm_asses']),f.lit('SEM ASSESSOR')))\
                          .when(prod['ds_segmento_agr'].isin('UNIVERSIDADES'),f.coalesce(f.trim(estr['nm_asses']),f.lit('SEM ASSESSOR')))\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'),f.coalesce(f.trim(estr['nm_asses']),f.lit('SEM ASSESSOR')))\
                          .otherwise(estr['nm_asses']).alias('nm_assessor_hist'),\
                         f.when((~(prod['cd_segmento'].cast('double').isin(15,227)) &\
                                (prod['ds_segmento_agr'].like('EMPRESAS%'))),f.coalesce(f.upper(f.trim(estr['modelo_atnd'])),f.lit('ASSESSORIA_PJ')))\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'),f.coalesce(f.upper(f.trim(estr['modelo_atnd'])),f.lit('ASSESSORIA_GI')))\
                          .when(prod['ds_segmento_agr'].isin('UNIVERSIDADES'),f.coalesce(f.upper(f.trim(estr['modelo_atnd'])),f.lit('ASSESSORIA_UNIV')))\
                           .otherwise(f.upper(estr['modelo_atnd'])).alias('modelo_atnd_hist'),\
                         du['du_br'].alias('du_br_hist'),\
                         f.when(prod['cd_segmento'].cast('int').isin('1','2','3','304'),'VAN GOGH')\
                          .when(prod['cd_segmento'].cast('int').isin('4','106'),'ESPECIAL')\
                          .when(prod['cd_segmento'].cast('int').isin('5','6','7'),'CLASSICO')\
                          .when(prod['cd_segmento'].cast('int').isin('9'),'SELECT')\
                          .when(prod['cd_segmento'].cast('int').isin('11','15','226'),'EMP 1')\
                          .when(prod['cd_segmento'].cast('int').isin('227'),'MEI')\
                          .when(prod['cd_segmento'].cast('int').isin('12','229'),'EMP 2')\
                          .when(prod['cd_segmento'].cast('int').isin('228'),'EMP 3')\
                          .when(prod['cd_segmento'].cast('int').isin('17','111'),'GI')\
                          .when(prod['cd_segmento'].cast('int').isin('211','212','213','214'),'UNIV')\
                          .otherwise('OUTROS').alias('segmento_hist'),\
                         f.when(prod['ds_produto'].isin('ACOES','COMMODITIES','DIREITO E BONUS','ETF','TITULOS PRIVADOS','ALGO+'),'CORRETORA EX TD')\
                          .when(prod['ds_produto'].isin('TD'),'TESOURO DIRETO')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['cd_subp'].cast('double') == 1550 ), 'CRI')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['cd_subp'].cast('double') == 9500 ), 'CRA')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['cd_subp'].cast('double') == 5 ), 'DEBENTURE')\
                          .when(((prod['ds_familia_carga'] == 'FUNDOS') & (prod['fl_fundo_auto'] == '1')),'FUNDOS AUTOMATICOS')\
                          .when(((prod['ds_familia_carga'] == 'FUNDOS') & (prod['fl_fundo_auto'] == '0')),'FUNDOS NAO AUTOMATICOS')\
                          .when(prod['cd_subp'].cast('int').isin('276','583'),'LCI CANCELAVEL')\
                          .otherwise(prod['ds_produto']).alias('produto_pla_hist'),\
                         prod['fl_diversificacao'].alias('fl_diversificacao_hist')\
                        )

In [ ]:
## PULSO HISTORICO
pulso_f_hist = pulso_hist.filter((f.col('ds_segmento_hist').isin('ESPECIAL','VAN GOGH','SELECT',\
                                                                 'EMPRESAS 1','EMPRESAS 2','EMPRESAS 3',\
                                                                 'GI','UNIVERSIDADES','MEI')) | \
                                 (f.col('modelo_atnd_hist') == 'AAA'))\
                .groupby('dt_anomes_hist',\
                        'du_br_hist',\
                        'dt_cver_movi_hist',\
                        'ds_produto_hist',\
                        'ds_segmento_hist',\
                        'pf_pj_hist',\
                        'produto_hist',\
                        'rede_hist',\
                        'regional_hist',\
                        'pv_hist',\
                        'clie_assessoria_hist',\
                        'fl_core_hist',\
                        'nm_coord_hist',\
                        'nm_tl_hist',\
                        'nm_assessor_hist',\
                        'modelo_atnd_hist',\
                        'segmento_hist',\
                        'produto_pla_hist',\
                        'fl_diversificacao_hist'\
                                 )\
               .agg(f.sum('aplicacao').alias('vl_aplicacao_hist'),\
                    f.sum('resgate').alias('vl_resgate_hist'),\
                    f.sum('vl_movi').alias('vl_cap_liquida_hist')\
                    ) 

In [ ]:
pulso_f_hist.registerTempTable("pulso_f_hist")

In [ ]:
pulso_hist = pulso_f_hist.select(
                          'du_br_hist',\
                          'dt_cver_movi_hist',\
                          'ds_produto_hist',\
                          'ds_segmento_hist',\
                          'pf_pj_hist',\
                          'produto_hist',\
                          f.when(f.col('rede_hist').isNull(),f.lit('SEM REDE'))\
                           .when(f.isnan('rede_hist'),f.lit('SEM REDE'))\
                           .when(f.regexp_replace(f.col('rede_hist'),' ','') == '',f.lit('SEM REDE'))\
                            .otherwise(f.trim(f.col('rede_hist'))).alias('rede_hist'),\
                          f.when(f.trim(f.col('regional_hist')) == '',f.lit('SEM REGIONAL'))\
                           .when(f.isnan('regional_hist'),f.lit('SEM REGIONAL'))\
                           .when(f.regexp_replace(f.col('regional_hist'),' ','') == '',f.lit('SEM REGIONAL'))\
                            .otherwise(f.trim(f.col('regional_hist'))).alias('regional_hist'),\
                          f.when(f.trim(f.col('pv_hist')) == '',f.lit('SEM PV'))\
                           .when(f.isnan('pv_hist'),f.lit('SEM PV'))\
                           .when(f.regexp_replace(f.col('pv_hist'),' ','') == '',f.lit('SEM PV'))\
                            .otherwise(f.trim(f.col('pv_hist'))).alias('pv_hist'),\
                          'clie_assessoria_hist',\
                          'fl_core_hist',\
                          f.when(f.trim(f.col('nm_coord_hist')) == '',f.lit('ALEXANDRE NOGUEIRA DE SOUZA'))\
                           .when(f.isnan('nm_coord_hist'),f.lit('ALEXANDRE NOGUEIRA DE SOUZA'))\
                           .when(f.regexp_replace(f.col('nm_coord_hist'),' ','') == '',f.lit('ALEXANDRE NOGUEIRA DE SOUZA'))\
                            .otherwise(f.trim(f.col('nm_coord_hist'))).alias('nm_coord_hist'),\
                          'nm_tl_hist',\
                          f.when(f.trim(f.col('nm_assessor_hist')) == '',f.lit('SEM ASSESSOR'))\
                            .when(f.isnan('nm_assessor_hist'),f.lit('SEM ASSESSOR'))\
                            .when(f.regexp_replace(f.col('nm_assessor_hist'),' ','') == '',f.lit('SEM ASSESSOR'))\
                            .otherwise(f.trim(f.col('nm_assessor_hist'))).alias('nm_assessor_hist'),\
                          f.when(f.trim(f.col('modelo_atnd_hist')) == '',f.lit('CARTEIRIZADO PJ'))\
                           .when(f.isnan('modelo_atnd_hist'),f.lit('CARTEIRIZADO PJ'))\
                           .when(f.regexp_replace(f.col('modelo_atnd_hist'),' ','') == '',f.lit('CARTEIRIZADO PJ'))\
                            .otherwise(f.trim(f.col('modelo_atnd_hist'))).alias('modelo_atnd_hist'),\
                          'segmento_hist',\
                          'produto_pla_hist',\
                          'fl_diversificacao_hist',\
                          'vl_aplicacao_hist',\
                          'vl_resgate_hist',\
                          'vl_cap_liquida_hist',\
                          'dt_anomes_hist'\
                         )

In [ ]:
pulso_hist.registerTempTable("pulso_hist")

In [ ]:
spark.sql("ALTER TABLE sand_invest.tb_pulso_qlik_histv4 DROP IF EXISTS PARTITION(dt_anomes_hist = {})".format(dt_anomes_aaaamm))

In [ ]:
spark.sql("""
insert into sand_invest.tb_pulso_qlik_histv4 
select * from pulso_hist
""")

### CASH

In [ ]:
if dia_util == '10':

        nme = spark.read.table('db_nme.nme_mensal_histo')\
                   .filter((f.col('data_particao').substr(1,6) == dt_nme_mens) &\
                           (f.col('codigo_linha_bp').isin('0230','0226','0221')) &\
                           (f.col('area_de_negocio_reporting').isin('BP - EMPRESAS MEI','BP - EMPRESAS 1','BP - EMPRESAS 2','BP - EMPRESAS 3',\
                                                                    'BP - VAN GOGH','BP - PF1','BP - PF2','BP - SELECT',\
                                                                    'BP - UNIVERSIDADES','BP - GOVERNOS E INSTITUIÇÕES')) &\
                           (f.col('indicador_pool')  == 'N'))


        cli = spark.read.table('sand_invest.tb_clientes_lake_new')\
            .filter(f.col('dt_anomes') == dt_nme_mens)
        
        
        cash = nme.join(cli,nme['penumper'] == cli['penumper'],'left')\
                  .groupby(f.when(f.col('area_de_negocio_reporting').isin('BP - PF1','BP - PF2'), 'ESPECIAL')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - VAN GOGH'), 'VAN GOGH')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - SELECT'), 'SELECT')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS 1'), 'EMPRESAS 1')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS MEI'), 'MEI')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS 2'), 'EMPRESAS 2')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS 3'), 'EMPRESAS 3')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - UNIVERSIDADES'), 'UNIVERSIDADES')\
                            .when(f.col('area_de_negocio_reporting').isin('BP - GOVERNOS E INSTITUIÇÕES'), 'GI')\
                             .otherwise('OUTROS').alias('ds_segmento'),\
                           f.coalesce(f.trim(cli['rede']),f.lit('SEM REDE')).alias('ds_rede'),\
                           f.coalesce(f.trim(cli['regional']),f.lit('SEM REGIONAL')).alias('ds_regional'),\
                           f.coalesce(f.trim(cli['pv']),f.lit('SEM PV')).alias('ds_pv'),\
                           f.when(f.col('codigo_linha_bp').isin('0230','0226'), 'CONTAMAX')\
                            .when(f.col('codigo_linha_bp').isin('0221'), 'DAV')\
                             .otherwise('OUTROS').alias('ds_produto'),\
                           f.col('data_particao').alias('dat_ref'),\
                           f.col('data_particao').substr(1,6).alias('dt_anomes')\
                          )\
                  .agg(f.sum('saldo_ponta').alias('saldo_cash'))

        tb_cash = cash.select('ds_segmento',\
                              'ds_rede',\
                              'ds_regional',\
                              'ds_pv',\
                              'ds_produto',\
                              'dat_ref',\
                              'saldo_cash',\
                              'dt_anomes'\
                            )

        tb_cash.registerTempTable("tmpcash")

        spark.sql("ALTER TABLE sand_invest.tb_qlik_cash DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_nme_mens))

        spark.sql("""
        insert into sand_invest.tb_qlik_cash
        select * from tmpcash
        """)

else:
    print('nme_diar')

In [ ]:
nme = spark.read.table('db_nme.nme_diaria_histo')\
           .filter((f.col('data_extracao') == dt_nme3) &\
                   (f.col('codigo_linha_bp').isin('0230','0226','0221')) &\
                   (f.col('area_de_negocio_reporting').isin('BP - EMPRESAS MEI','BP - EMPRESAS 1','BP - EMPRESAS 2','BP - EMPRESAS 3',\
                                                            'BP - VAN GOGH','BP - PF1','BP - PF2','BP - SELECT',\
                                                            'BP - UNIVERSIDADES','BP - GOVERNOS E INSTITUIÇÕES')) &\
                   (f.col('indicador_pool')  == 'N'))

In [ ]:
cash = nme.join(cli,nme['penumper'] == cli['penumper'],'left')\
          .groupby(f.when(f.col('area_de_negocio_reporting').isin('BP - PF1','BP - PF2'), 'ESPECIAL')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - VAN GOGH'), 'VAN GOGH')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - SELECT'), 'SELECT')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS 1'), 'EMPRESAS 1')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS MEI'), 'MEI')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS 2'), 'EMPRESAS 2')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - EMPRESAS 3'), 'EMPRESAS 3')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - UNIVERSIDADES'), 'UNIVERSIDADES')\
                    .when(f.col('area_de_negocio_reporting').isin('BP - GOVERNOS E INSTITUIÇÕES'), 'GI')\
                     .otherwise('OUTROS').alias('ds_segmento'),\
                   f.coalesce(f.trim(cli['rede']),f.lit('SEM REDE')).alias('ds_rede'),\
                   f.coalesce(f.trim(cli['regional']),f.lit('SEM REGIONAL')).alias('ds_regional'),\
                   f.coalesce(f.trim(cli['pv']),f.lit('SEM PV')).alias('ds_pv'),\
                   f.when(f.col('codigo_linha_bp').isin('0230','0226'), 'CONTAMAX')\
                    .when(f.col('codigo_linha_bp').isin('0221'), 'DAV')\
                     .otherwise('OUTROS').alias('ds_produto'),\
                   f.col('data_extracao').alias('dat_ref'),\
                   f.col('data_extracao').substr(1,6).alias('dt_anomes')\
                  )\
          .agg(f.sum('saldo_ponta').alias('saldo_cash'))

In [ ]:
tb_cash = cash.select('ds_segmento',\
                      'ds_rede',\
                      'ds_regional',\
                      'ds_pv',\
                      'ds_produto',\
                      'dat_ref',\
                      'saldo_cash',\
                      'dt_anomes'\
                    )

In [ ]:
tb_cash.registerTempTable("tmpcash")

In [ ]:
spark.sql("ALTER TABLE sand_invest.tb_qlik_cash DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_anomes_nme))

In [ ]:
spark.sql("""
insert into sand_invest.tb_qlik_cash
select * from tmpcash
""")

###  TOP 10 - DIARIO

In [ ]:
###  TOP 10
prod = spark.read.table('sand_invest.tb_prod_lake_new')\
            .filter((f.col('dt_anomes') == dt_anomes_aaaamm) &\
                    ((f.col('fl_prod') == '1') | (f.col('ds_produto') == 'FUNDOS')) &\
                    (f.col('ds_segmento_agr').isin('CLASSICO','ESPECIAL','VAN_GOGH','SELECT',\
                                                    'EMPRESAS_1','EMPRESAS_2','EMPRESAS_3',\
                                                    'GOVERNOS','INSTITUICOES','UNIVERSIDADES'))\
                   )

In [ ]:
### TOP10 DIAR
topdiar = prod.join(cli,(prod['nr_pess'] == cli['penumper']),'left')\
                 .join(anb,prod['cd_fundo'].cast('float') == anb['cd_mx'].cast('float'),'left')\
                 .join(du, (prod['dt_cver_movi'] == du['data']),'left')\
                 .select(prod['dt_anomes'].alias('dt_anomes_diar'),\
                         prod['dt_cver_movi'].alias('dt_cver_movi_diar'),\
                         prod['nr_pess'].alias('nr_pess_diar'),\
                         cli['nm_clie'].alias('nm_clie_diar'),\
                         f.when(prod['ds_produto'].isin('ACOES','COMMODITIES','DIREITO E BONUS','ETF','TITULOS PRIVADOS','SWAP'),'PRODUTOS BOLSA')\
                          .when(prod['ds_produto'] == 'ALGO+', 'ALGO +')\
                          .when(((prod['ds_familia_carga'] == 'FUNDOS') & (prod['fl_fundo_auto'] == '1')),'FUNDOS AUTOMATICOS')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & ((f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               (anb['ds_class_anbima'] == '0')),'FUNDOS RF')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & (~(f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               ~(anb['ds_class_anbima'] == '0')),'FUNDOS ESTRUTURADOS')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['ds_subp'] == 'CRI'), 'CRI')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['ds_subp'] == 'CRA'), 'CRA')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['ds_subp'] == 'DEBÊNTURES'), 'DEBENTURE')\
                          .when((prod['ds_produto'] == 'PUBLICO'), 'TITULOS PUBLICOS')\
                          .when(prod['ds_familia_carga'] == 'PUBLICO', 'TITULOS PUBLICOS')\
                          .when(prod['ds_familia_carga'] == 'PREVIDENCIA', 'PREVIDENCIA')\
                          .when(prod['ds_produto'] == 'COM', 'COMPROMISSADAS')\
                          .otherwise(prod['ds_produto']).alias('ds_produto_diar'),\
                         f.when(prod['ds_segmento_agr'].isin('CLASSICO','ESPECIAL'), 'ESPECIAL')\
                          .when(prod['ds_segmento_agr'] == 'VAN_GOGH', 'VAN GOGH')\
                          .when(prod['cd_segmento'].cast('double').isin(15,227),'MEI')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_1', 'EMPRESAS 1')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_2', 'EMPRESAS 2')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_3', 'EMPRESAS 3')\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'), 'GI')\
                           .otherwise(prod['ds_segmento_agr']).alias('ds_segmento_diar'),\
                         f.when(prod['ds_segmento_agr'].isin('SELECT','VAN_GOGH','ESPECIAL','CLASSICO'),'PF')\
                          .when(prod['ds_segmento_agr'].isin('EMPRESAS_1','EMPRESAS_2','EMPRESAS_3'),'EMPRESAS')\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES','UNIVERSIDADES'),'GI/UNI').alias('PF_PJ_diar'),\
                         f.when((prod['tp_movi'] == '1'),prod['vl_movi'].cast('double'))\
                                     .otherwise(0).alias('aplicacao_diar'),\
                         f.when((prod['tp_movi'] == '-1'),f.abs(prod['vl_movi'].cast('double')))\
                                      .otherwise(0).alias('resgate_diar'),\
                         f.coalesce(f.trim(cli['rede']),f.lit('SEM REDE')).alias('rede_diar'),\
                         f.when((~(prod['cd_segmento'].cast('double').isin(15,227)) &\
                                (prod['ds_segmento_agr'].like('EMPRESAS%'))),f.coalesce(f.upper(f.trim(cli['modelo_atnd'])),f.lit('ASSESSORIA_PJ')))\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'),f.coalesce(f.upper(f.trim(cli['modelo_atnd'])),f.lit('ASSESSORIA_GI')))\
                          .when(prod['ds_segmento_agr'].isin('UNIVERSIDADES'),f.coalesce(f.upper(f.trim(cli['modelo_atnd'])),f.lit('ASSESSORIA_UNIV')))\
                           .otherwise(f.upper(cli['modelo_atnd'])).alias('modelo_atnd_diar'),\
                         du['du_br'].alias('du_br_diar'))\
                 .withColumn('rank_dt_prod',f.row_number().over(Window.partitionBy('dt_cver_movi_diar').orderBy(f.col('aplicacao_diar').cast('double').desc())))\
                 .withColumn('rank_dt_resg',f.row_number().over(Window.partitionBy('dt_cver_movi_diar').orderBy(f.col('resgate_diar').cast('double').desc())))\
                 .withColumn('rank_rede_prod',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','rede_diar').orderBy(f.col('aplicacao_diar').cast('double').desc())))\
                 .withColumn('rank_rede_resg',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','rede_diar').orderBy(f.col('resgate_diar').cast('double').desc())))\
                 .withColumn('rank_prod_prod',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','ds_produto_diar').orderBy(f.col('aplicacao_diar').cast('double').desc())))\
                 .withColumn('rank_prod_resg',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','ds_produto_diar').orderBy(f.col('resgate_diar').cast('double').desc())))\
                 .withColumn('rank_seg_prod',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','ds_segmento_diar').orderBy(f.col('aplicacao_diar').cast('double').desc())))\
                 .withColumn('rank_seg_resg',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','ds_segmento_diar').orderBy(f.col('resgate_diar').cast('double').desc())))\
                 .withColumn('rank_mol_prod',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','modelo_atnd_diar').orderBy(f.col('aplicacao_diar').cast('double').desc())))\
                 .withColumn('rank_mol_resg',f.row_number().over(Window.partitionBy('dt_cver_movi_diar','modelo_atnd_diar').orderBy(f.col('resgate_diar').cast('double').desc())))\
                 .filter((f.col('rank_dt_prod') < 11) | (f.col('rank_dt_resg') < 11) | \
                         (f.col('rank_rede_prod') < 11) | (f.col('rank_rede_resg') < 11) | \
                         (f.col('rank_prod_prod') < 11) | (f.col('rank_prod_resg') < 11) | \
                         (f.col('rank_seg_prod') < 11) | (f.col('rank_seg_resg') < 11) | \
                         (f.col('rank_mol_prod') < 11) | (f.col('rank_mol_resg') < 11)\
                        )

In [ ]:
topdiar.registerTempTable("topdiar")

In [ ]:
top_diar = spark.read.table("topdiar")\
               .select(
                       'dt_cver_movi_diar',\
                       'nr_pess_diar',\
                       'nm_clie_diar',\
                       'ds_produto_diar',\
                       'ds_segmento_diar',\
                       'PF_PJ_diar',\
                       'aplicacao_diar',\
                       'resgate_diar',\
                       'rede_diar',\
                       'modelo_atnd_diar',\
                       'du_br_diar',\
                       'dt_anomes_diar'\
                      )

In [ ]:
top_diar.registerTempTable("top_diar")

In [ ]:
spark.sql("ALTER TABLE sand_invest.tb_pulso_qlik_topdiar DROP IF EXISTS PARTITION(dt_anomes_diar = {})".format(dt_anomes_aaaamm))

In [ ]:
spark.sql("""
insert into sand_invest.tb_pulso_qlik_topdiar
select * from top_diar
""")

### TOP 10 - MENSAL

In [ ]:
### TOP10 MENS
topmens = prod.join(cli,(prod['nr_pess'] == cli['penumper']),'left')\
              .join(anb,prod['cd_fundo'].cast('float') == anb['cd_mx'].cast('float'),'left')\
              .join(du, (prod['dt_cver_movi'] == du['data']),'left')\
              .select(prod['dt_anomes'].alias('dt_anomes_mens'),\
                      prod['dt_cver_movi'].alias('dt_cver_movi_mens'),\
                      prod['nr_pess'].alias('nr_pess_mens'),\
                      cli['nm_clie'].alias('nm_clie_mens'),\
                      f.when(prod['ds_produto'].isin('ACOES','COMMODITIES','DIREITO E BONUS','ETF','TITULOS PRIVADOS','SWAP'),'PRODUTOS BOLSA')\
                          .when(prod['ds_produto'] == 'ALGO+', 'ALGO +')\
                          .when(((prod['ds_familia_carga'] == 'FUNDOS') & (prod['fl_fundo_auto'] == '1')),'FUNDOS AUTOMATICOS')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & ((f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               (anb['ds_class_anbima'] == '0')),'FUNDOS RF')\
                          .when((prod['ds_familia_carga'] == 'FUNDOS') & (~(f.coalesce(anb['ds_class_anbima'],f.lit('Renda Fixa')).like('%Renda Fixa%')) | \
                                                                               ~(anb['ds_class_anbima'] == '0')),'FUNDOS ESTRUTURADOS')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['ds_subp'] == 'CRI'), 'CRI')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['ds_subp'] == 'CRA'), 'CRA')\
                          .when((prod['ds_produto'] == 'TESOURARIA  SIGOM') & (prod['ds_subp'] == 'DEBÊNTURES'), 'DEBENTURE')\
                          .when((prod['ds_produto'] == 'PUBLICO'), 'TITULOS PUBLICOS')\
                          .when(prod['ds_familia_carga'] == 'PUBLICO', 'TITULOS PUBLICOS')\
                          .when(prod['ds_familia_carga'] == 'PREVIDENCIA', 'PREVIDENCIA')\
                          .when(prod['ds_produto'] == 'COM', 'COMPROMISSADAS')\
                          .otherwise(prod['ds_produto']).alias('ds_produto_mens'),\
                      f.when(prod['ds_segmento_agr'].isin('CLASSICO','ESPECIAL'), 'ESPECIAL')\
                          .when(prod['ds_segmento_agr'] == 'VAN_GOGH', 'VAN GOGH')\
                          .when(prod['cd_segmento'].cast('double').isin(15,227),'MEI')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_1', 'EMPRESAS 1')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_2', 'EMPRESAS 2')\
                          .when(prod['ds_segmento_agr'] == 'EMPRESAS_3', 'EMPRESAS 3')\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'), 'GI')\
                           .otherwise(prod['ds_segmento_agr']).alias('ds_segmento_mens'),\
                      f.when(prod['ds_segmento_agr'].isin('SELECT','VAN_GOGH','ESPECIAL','CLASSICO'),'PF')\
                          .when(prod['ds_segmento_agr'].isin('EMPRESAS_1','EMPRESAS_2','EMPRESAS_3'),'EMPRESAS')\
                          .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES','UNIVERSIDADES'),'GI/UNI').alias('PF_PJ_mens'),\
                      f.when(prod['ds_segmento_agr'].like('EMPRESAS%'),f.lit('1'))
                       .otherwise(f.coalesce(cli['clie_assessoria'],f.lit('0'))).alias('clie_assessoria_mens'),\
                      f.when((prod['tp_movi'] == '1'),prod['vl_movi'].cast('double'))\
                       .otherwise(0).cast('int').alias('aplicacao_mens'),\
                      f.when((prod['tp_movi'] == '-1'),f.abs(prod['vl_movi'].cast('double')))\
                       .otherwise(0).alias('resgate_mens'),\
                      f.when(f.trim(cli['rede']) == '',f.lit('SEM REDE'))\
                       .otherwise(cli['rede']).alias('rede_mens'),\
                      f.when((~(prod['cd_segmento'].cast('double').isin(15,227)) &\
                               (prod['ds_segmento_agr'].like('EMPRESAS%'))),f.coalesce(f.upper(f.trim(cli['modelo_atnd'])),f.lit('ASSESSORIA_PJ')))\
                       .when(prod['ds_segmento_agr'].isin('GOVERNOS','INSTITUICOES'),f.coalesce(f.upper(f.trim(cli['modelo_atnd'])),f.lit('ASSESSORIA_GI')))\
                       .when(prod['ds_segmento_agr'].isin('UNIVERSIDADES'),f.coalesce(f.upper(f.trim(cli['modelo_atnd'])),f.lit('ASSESSORIA_UNIV')))\
                        .otherwise(f.upper(cli['modelo_atnd'])).alias('modelo_atnd_mens'),\
                      du['du_br'].alias('du_br_mens'))\
                    .withColumn('rank_dt_prod',f.row_number().over(Window.partitionBy('dt_anomes_mens').orderBy(f.col('aplicacao_mens').cast('double').desc())))\
                    .withColumn('rank_dt_resg',f.row_number().over(Window.partitionBy('dt_anomes_mens').orderBy(f.col('resgate_mens').cast('double').desc())))\
                    .withColumn('rank_rede_prod',f.row_number().over(Window.partitionBy('dt_anomes_mens','rede_mens').orderBy(f.col('aplicacao_mens').cast('double').desc())))\
                    .withColumn('rank_rede_resg',f.row_number().over(Window.partitionBy('dt_anomes_mens','rede_mens').orderBy(f.col('resgate_mens').cast('double').desc())))\
                    .withColumn('rank_prod_prod',f.row_number().over(Window.partitionBy('dt_anomes_mens','ds_produto_mens').orderBy(f.col('aplicacao_mens').cast('double').desc())))\
                    .withColumn('rank_prod_resg',f.row_number().over(Window.partitionBy('dt_anomes_mens','ds_produto_mens').orderBy(f.col('resgate_mens').cast('double').desc())))\
                    .withColumn('rank_seg_prod',f.row_number().over(Window.partitionBy('dt_anomes_mens','ds_segmento_mens').orderBy(f.col('aplicacao_mens').cast('double').desc())))\
                    .withColumn('rank_seg_resg',f.row_number().over(Window.partitionBy('dt_anomes_mens','ds_segmento_mens').orderBy(f.col('resgate_mens').cast('double').desc())))\
                    .withColumn('rank_mol_prod',f.row_number().over(Window.partitionBy('dt_anomes_mens','modelo_atnd_mens').orderBy(f.col('aplicacao_mens').cast('double').desc())))\
                    .withColumn('rank_mol_resg',f.row_number().over(Window.partitionBy('dt_anomes_mens','modelo_atnd_mens').orderBy(f.col('resgate_mens').cast('double').desc())))\
                    .filter((f.col('rank_dt_prod') < 11) | (f.col('rank_dt_resg') < 11) | \
                            (f.col('rank_rede_prod') < 11) | (f.col('rank_rede_resg') < 11) | \
                            (f.col('rank_prod_prod') < 11) | (f.col('rank_prod_resg') < 11) | \
                            (f.col('rank_seg_prod') < 11) | (f.col('rank_seg_resg') < 11) | \
                            (f.col('rank_mol_prod') < 11) | (f.col('rank_mol_resg') < 11)\
                           )\

In [ ]:
spark.sql("ALTER TABLE sand_invest.topmens_spark DROP IF EXISTS PARTITION(dt_anomes_mens = {})".format(dt_anomes_aaaamm))

In [ ]:
try:
    topmens.write.mode("append").partitionBy('dt_anomes_mens').saveAsTable('sand_invest.topmens_spark')
except:
    print("topmens não subiu")

### PPTO

#### PPTO SEGMENTO PRODUTO

In [ ]:
prod = spark.read.table('sand_invest.tb_prod_lake_new')\
            .filter((f.col('dt_anomes') == dt_anomes_aaaamm) &\
                    (f.col('fl_prod') == '1') &\
                    (f.col('ds_segmento_agr').isin('CLASSICO','ESPECIAL','SELECT','VAN_GOGH',\
                                                   'EMPRESAS_1','EMPRESAS_2','EMPRESAS_3',\
                                                   'GOVERNOS','INSTITUICOES','UNIVERSIDADES')))\
            .groupby(f.when((f.col('ds_familia_carga') == 'CORRETORA') & (~(f.col('ds_produto') == 'TESOURO DIRETO')), 'BOLSA')\
                      .when((f.col('ds_familia_carga').isin('FUNDOS','PREVIDENCIA','POUPANCA')), f.col('ds_familia_carga'))
                      .when((f.col('ds_subp').isin('CRA','CRI')), f.col('ds_subp'))\
                      .when((f.col('ds_subp') == 'DEBÊNTURES'), 'DEBENTURE')
                      .when((f.col('ds_produto') == 'COM'), 'COMPROMISSADA')\
                      .when((f.col('ds_produto') == 'PUBLICO'), 'TITULOS PUBLICOS')\
                      .when((f.col('ds_produto') == 'ATIVOS FINANCEI'), 'LIG')\
                       .otherwise(f.col('ds_produto')).alias('ds_produto'),\
                     f.when((f.col('ds_segmento_agr').isin('CLASSICO','ESPECIAL')), 'ESPECIAL')\
                      .when((f.col('cd_segmento').cast('double').isin(15,227)), 'MEI')\
                      .when((f.col('ds_segmento_agr').isin('GOVERNOS','INSTITUICOES')), 'GI')\
                      .when((f.col('ds_segmento_agr').like('EMPRESAS%')) | \
                            (f.col('ds_segmento_agr') == 'VAN_GOGH'), f.regexp_replace(f.col('ds_segmento_agr'),'_', ' '))\
                       .otherwise(f.col('ds_segmento_agr')).alias('ds_segmento'),\
                     f.col('dt_anomes')\
                     )\
            .agg(f.sum(f.when((f.col('tp_movi') == '1'), f.col('vl_movi'))\
                         .otherwise(0)).alias('cap_bruta'),\
                 f.sum(f.when((f.col('tp_movi') == '-1'), f.col('vl_movi'))\
                         .otherwise(0)).alias('resgate'),\
                 f.sum(f.col('vl_movi')).alias('cap_liq')\
                )

In [ ]:
ppto = spark.read.table('sand_invest.tb_ppto_202203')\
            .groupby(f.when(f.col('ds_segmento_agr').isin('CLASSICO','ESPECIAL'), 'ESPECIAL')\
                      .when(f.col('ds_segmento_agr').like('EMP%'), f.regexp_replace(f.col('ds_segmento_agr'),'EMP','EMPRESAS'))\
                      .when(f.col('ds_segmento_agr') == 'UNIV', 'UNIVERSIDADES')\
                        .otherwise(f.col('ds_segmento_agr')).alias('segmento'),\
                     f.when(f.col('ds_produto') == 'TD', 'TESOURO DIRETO')\
                      .when(f.col('ds_produto') == 'TITULOS PUBLICOS TESOURARIA', 'TITULOS PUBLICOS')\
                        .otherwise(f.col('ds_produto')).alias('produto'))\
            .agg(f.sum(f.col('valor').cast('double')).alias('cap_liquida'))

In [ ]:
base = prod.join(ppto, (prod['ds_segmento'] == ppto['segmento']) &\
                       (prod['ds_produto'] == ppto['produto']), 'left')\
           .groupby(f.coalesce(prod['ds_produto'],ppto['produto']).alias('ds_produto'),\
                    f.coalesce(prod['ds_segmento'],ppto['segmento']).alias('ds_segmento'),\
                    prod['dt_anomes']\
                   )\
           .agg(f.sum(f.coalesce(prod['cap_liq'],f.lit(0))).alias('cap_liq'),\
                f.sum(f.coalesce(ppto['cap_liquida'],f.lit(0))).alias('meta_liq')\
                )

In [ ]:
base_fim = base.select('ds_produto',\
                       'ds_segmento',\
                       'cap_liq',\
                       'meta_liq',\
                       f.col('dt_anomes').cast('string')
                      )

In [ ]:
#base_fim.registerTempTable('tmpppto')

In [ ]:
#spark.sql("ALTER TABLE sand_invest.tb_qlik_ppto_202203_v2 DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_anomes_aaaamm))

In [ ]:
#spark.sql("""

#insert into sand_invest.tb_qlik_ppto_202203_v2 
#select * from tmpppto
#
#""")

#### PPTO ASSESSOR

In [ ]:
cli = spark.read.table('sand_invest.tb_clientes_lake_new')\
                .filter((f.col('dt_anomes') == dt_anomes_aaaamm) &\
                        (f.col('pf_pj') == 'F'))

In [ ]:
prod = spark.read.table('sand_invest.tb_prod_lake_new')\
                 .filter((f.col('fl_prod') == '1') &\
                         (f.col('dt_anomes') == dt_anomes_aaaamm))

In [ ]:
pulso = prod.join(cli, f.lpad(cli['penumper'],8,'0') == f.lpad(prod['nr_pess'],8,'0'), 'left')\
                .groupby(cli['modelo_atnd'],\
                         cli['nm_coord'],\
                         cli['nm_tl'],\
                         cli['nm_assessor'],\
                         cli['dt_anomes']\
                        )\
                .agg(f.sum(prod['vl_movi']).alias('vl_movi'))\
                .filter(~cli['nm_assessor'].isNull())

In [ ]:
pulso = pulso.select('modelo_atnd',\
                     'nm_coord',\
                     'nm_tl',\
                     'nm_assessor',\
                     'vl_movi',\
                     f.lit('2000000').alias('meta'),\
                     'dt_anomes')\

In [ ]:
#pulso.registerTempTable('tmppptoasse')

In [ ]:
#spark.sql("ALTER TABLE sand_invest.tb_ref_assessor DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_anomes_aaaamm))

In [ ]:
#spark.sql("""
#insert into sand_invest.tb_ref_assessor
#select * from tmppptoasse
#
#""")

#### PPTO REDE 

In [ ]:
cli = spark.read.table('sand_invest.tb_clientes_lake_new')\
                .filter(f.col('dt_anomes') == dt_anomes_aaaamm)\

In [ ]:
prod = spark.read.table('sand_invest.tb_prod_lake_new')\
                 .filter((f.col('fl_prod') == '1') &\
                         (f.col('dt_anomes') == dt_anomes_aaaamm) &\
                         (f.col('ds_segmento_agr').isin('CLASSICO','ESPECIAL','VAN_GOGH','SELECT',\
                                                        'EMPRESAS_1','EMPRESAS_2','EMPRESAS_3',\
                                                        'GOVERNOS','INSTITUICOES','UNIVERSIDADES'))
                        )

In [ ]:
pulso = prod.join(cli, f.lpad(cli['penumper'],8,'0') == f.lpad(prod['nr_pess'],8,'0'), 'left')\
                .groupby(f.coalesce(cli['rede'],f.lit('SEM REDE')).alias('rede'),\
                         f.coalesce(cli['regional'],f.lit('SEM REGIONAL')).alias('regional'),\
                         f.coalesce(cli['pv'],f.lit('SEM PV')).alias('pv'),\
                         f.coalesce(cli['dt_anomes'],prod['dt_anomes']).alias('dt_anomes')\
                        )\
                .agg(f.sum(prod['vl_movi']).alias('vl_movi'))\
                .withColumn('count_pv', f.count(f.col('pv')).over(Window.partitionBy(f.col('dt_anomes'))))\
                .withColumn('meta', (f.lit(29680667330)/ \
                                    (f.col('count_pv'))))

In [ ]:
pulso = pulso.select('rede',\
                     'regional',\
                     'pv',\
                     'vl_movi',\
                     'meta',\
                     'dt_anomes'
                    )

In [ ]:
#pulso.registerTempTable('tmppptorede')

In [ ]:
#spark.sql("ALTER TABLE sand_invest.tb_ref_rede DROP IF EXISTS PARTITION(dt_anomes = {})".format(dt_anomes_aaaamm))

In [ ]:
#spark.sql("""
#insert into sand_invest.tb_ref_rede
#select * from tmppptorede
#
#""")

In [ ]:
spark.stop()